In [1]:
!pip install pymysql
import pymysql


Defaulting to user installation because normal site-packages is not writeable


In [2]:
from datetime import date, datetime

import numpy as np
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [3]:
# Establish a connection to the database
conn = pymysql.connect(
    host='192.168.1.27',
    user='fgichere',
    password='Burn@2020$',
    database='copkbi'
)

In [4]:
# Read registrations from the database into a pandas dataframe
df_registrations_survey = pd.read_sql_query('''
                                    SELECT *
                                    FROM copkbi.registrations_surveys
                                     WHERE submission_time >= '2023-08-31'AND COUNTRY IN ('DRC', 'KENYA', 'MOZAMBIQUE', 'NIGERIA', 'SOMALILAND', 'SOUTH SOMALIA', 'TANZANIA');''', conn)
df_registrations = df_registrations_survey

C:\Users\basil.chinedu\AppData\Local\Temp\10\ipykernel_55092\3616840694.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_registrations_survey = pd.read_sql_query('''


In [5]:
# Read registrations from the database into a pandas dataframe
df_verifications = pd.read_sql_query('''
                                    SELECT v.*, r.verified_cc, r.call_status, r.call_attempts
                                    FROM copkbi.verification_sampled_data v
                                    LEFT JOIN registrations_surveys r ON v.sn_mod = r.sn_mod
                                    WHERE v.submission_time >= '2023-08-31'
                                    AND v.COUNTRY IN ('DRC', 'KENYA', 'MOZAMBIQUE', 'NIGERIA', 'SOMALILAND', 'SOUTH SOMALIA', 'TANZANIA');
                                    ''', conn)

C:\Users\basil.chinedu\AppData\Local\Temp\10\ipykernel_55092\3681986337.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_verifications = pd.read_sql_query('''


In [6]:
##Remove duplicates
df_verifications.drop_duplicates(subset=['sn_mod', 'main_contact'], inplace=True)

##Remove duplicates
df_registrations.drop_duplicates(subset=['sn_mod', 'main_contact'], inplace=True)


In [7]:
df_verifications['call_attempts'] = df_verifications['call_attempts'].astype(str)

In [8]:
print("Columns in df_registrations:")
print(df_registrations.columns.tolist())

print("\nColumns in df_verifications:")
print(df_verifications.columns.tolist())


Columns in df_registrations:
['start', 'end', 'username', 'activator_label', 'survey_region', 'survey_channel', 'name', 'main_contact', 'alternative_phone_label', 'multi_ics', 'sn_mod', 'carbon_consent_label', 'primary_fuel', 'frequecy_of_cooking_other_fuel', 'respondents_gender', 'residence_level1', 'residence_level2', 'residence_level3', 'customer_address', 'gps_coordinates', 'payment_confirmation_code', 'submission_time', 'id', 'uuid', 'submitted_by', 'country', 'call_status', 'call_max_date', 'valid_sn_cc', 'sn_match', 'sn_cc', 'verified_cc', 'survey', 'original_case_uuid', 'otp_match', 'activator_id', 'customer_id', 'product_name', 'team_leader', 'territory_sales_manager', 'distributor', 'ship_date', 'carbon_eligible', 'team', 'valid_sn', 'unique_sn', 'valid_fuel', 'reg_after_ship_date', 'dist_carbon_eligible', 'sn_assigned', 'phone_assigned', 'registered_sn', 'poa_cpa', 'valid', 'error_text', 'send_error_to', 'creditable', 'reason_for_non_creditability', 'preferred_time_to_be_cal

In [9]:
##ALL REGISTRATIONS
# Calculate distinct count of main contacts from df_registrations
total_registrations = df_registrations['main_contact'].nunique()

# Summarize registrations by country
import pandas as pd

# Define a custom function to calculate total calls made
def total_calls_made_func(series):
    return series.loc[~series.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED'])].replace('None', '0').astype(int).sum()

# Summarize registrations by country
summary_registrations = df_verifications.groupby('country').agg(
    sample_pasted=('main_contact', 'nunique'),  # Using 'nunique' directly as an aggregation function
    verified_registrations=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    partially_verified=('verified_cc', lambda x: x.isin(['PARTIALLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(df_verifications['verified_cc'] != 'NOT CONTACTED') & (df_verifications['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_calls_made=('call_attempts', total_calls_made_func),  # Using custom aggregation function
    customers_reached=('call_status', lambda x: x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT']).sum()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA',
                                                     'PARTIALLY VERIFIED', 
                                                     'PARTIALLY VERIFIED - DORMANT', 
                                                     'DORMANT']).sum())
).reset_index()


# Rename the count columns for clarity
summary_registrations.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Calculate total registrations by country
total_registrations_by_country = df_registrations.groupby('country')['main_contact'].nunique().reset_index()
total_registrations_by_country.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Merge total registrations by country with summary_registrations
summary_registrations = pd.merge(summary_registrations, total_registrations_by_country, on='country', how='left')

# Calculate verification rate
summary_registrations['old_verification_rate'] = (summary_registrations['verified_registrations'] / summary_registrations['total_verified']) * 100

# Add "%" symbol to verification_rate column
summary_registrations['old_verification_rate'] = summary_registrations['old_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage correctly
summary_registrations['sample_percentage'] = (summary_registrations['sample_pasted'] / summary_registrations['total_registrations']) * 100

# Add "%" symbol to sample_percentage column
summary_registrations['sample_percentage'] = summary_registrations['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call rate correctly
summary_registrations['call_rate'] = (summary_registrations['customer_called'] / summary_registrations['sample_pasted']) * 100

# Add "%" symbol to call_rate column
summary_registrations['call_rate'] = summary_registrations['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate correctly
summary_registrations['reachability_rate'] = (summary_registrations['customers_reached'] / summary_registrations['customer_called']) * 100

# Add "%" symbol to reachability_rate column
summary_registrations['reachability_rate'] = summary_registrations['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate correctly
summary_registrations['percentage_fully_verified'] = (summary_registrations['fully_verified'] / summary_registrations['sample_pasted']) * 100

# Add "%" symbol to fully verified_rate column
summary_registrations['percentage_fully_verified'] = summary_registrations['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification rate correctly
summary_registrations['new_verification_rate'] = (summary_registrations['fully_verified'] / summary_registrations['customer_called']) * 100

# Add "%" symbol to new_verification rate column
summary_registrations['new_verification_rate'] = summary_registrations['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Add a row to the summary DataFrame for total registrations across all countries
total_row = pd.DataFrame({'country': ['Total'],
                          'total_registrations': [summary_registrations['total_registrations'].sum()],
                          'sample_pasted': [summary_registrations['sample_pasted'].sum()],
                          'verified_registrations': [summary_registrations['verified_registrations'].sum()],
                          'fully_verified': [summary_registrations['fully_verified'].sum()],
                          'partially_verified': [summary_registrations['partially_verified'].sum()],
                          'customer_called': [summary_registrations['customer_called'].sum()],
                          'total_calls_made': [summary_registrations['total_calls_made'].sum()],
                          'customers_reached': [summary_registrations['customers_reached'].sum()],
                          'total_verified': [summary_registrations['total_verified'].sum()]})

# Calculate total verification rate
old_verification_rate = (total_row['verified_registrations'] / total_row['total_verified']) * 100

# Add total verification rate to the total row
total_row['old_verification_rate'] = old_verification_rate.apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage for total row
total_row['sample_percentage'] = (total_row['sample_pasted'] / total_row['total_registrations']) * 100

# Add "%" symbol to sample percentage column in the total row
total_row['sample_percentage'] = total_row['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call_rate for total row
total_row['call_rate'] = (total_row['customer_called'] / total_row['sample_pasted']) * 100

# Add "%" symbol to call_rate column in the total row
total_row['call_rate'] = total_row['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate for total row
total_row['reachability_rate'] = (total_row['customers_reached'] / total_row['customer_called']) * 100

# Add "%" symbol to reachability_rate column in the total row
total_row['reachability_rate'] = total_row['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate for total row
total_row['percentage_fully_verified'] = (total_row['fully_verified'] / total_row['sample_pasted']) * 100

# Add "%" symbol to fully verified column in the total row
total_row['percentage_fully_verified'] = total_row['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification_rate for total row
total_row['new_verification_rate'] = (total_row['fully_verified'] / total_row['customer_called']) * 100

# Add "%" symbol to new_verification_rate column in the total row
total_row['new_verification_rate'] = total_row['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Concatenate summary registrations with total row
summary_registrations = pd.concat([summary_registrations, total_row], ignore_index=True)

# Assuming your DataFrame is named summary_registrations

# Selecting the desired columns
selected_columns = ['total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'old_verification_rate', 'new_verification_rate', 'customer_called','total_calls_made', 'call_rate', 'customers_reached', 'reachability_rate', 'fully_verified', 'partially_verified', 'percentage_fully_verified']


# Adding the 'country' column back as index
summary_registrations.reset_index(inplace=True)

# Reordering the columns to include 'country' as the first column
summary_registrations = summary_registrations[['country'] + selected_columns]

# Adding a title to the DataFrame
summary_registrations_title = "September 2023 to February 2024 Registrations"
summary_registrations = pd.concat([pd.DataFrame([summary_registrations_title], columns=['Title']), summary_registrations],
                                         ignore_index=True)

# Displaying the DataFrame
print(summary_registrations)






                                           Title        country  \
0  September 2023 to February 2024 Registrations            NaN   
1                                            NaN            DRC   
2                                            NaN          KENYA   
3                                            NaN     MOZAMBIQUE   
4                                            NaN        NIGERIA   
5                                            NaN     SOMALILAND   
6                                            NaN  SOUTH SOMALIA   
7                                            NaN       TANZANIA   
8                                            NaN          Total   

   total_registrations  sample_pasted sample_percentage  \
0                  NaN            NaN               NaN   
1              52454.0        26046.0            49.65%   
2              66898.0        27026.0            40.40%   
3              30270.0        18849.0            62.27%   
4             132541.0        4749

In [10]:
######################
##customers_not_reached = (~df_verifications['call_status'].isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT'])).sum()
#####################
##customers_reached = df_verifications['call_status'].isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT']).sum()

##print(customers_not_reached)

##print(customers_reached)



In [11]:
df_verifications['submission_time'] = pd.to_datetime(df_verifications['submission_time'])
df_registrations['submission_time'] = pd.to_datetime(df_registrations['submission_time'])


In [12]:
##september
# Filter DataFrame for submissions in September 2023
df_verifications_sep_2023 = df_verifications[df_verifications['submission_time'].dt.year == 2023]
df_verifications_sep_2023 = df_verifications_sep_2023[df_verifications_sep_2023['submission_time'].dt.month == 9]

# Filter DataFrame for registrations in September 2023
df_registrations_sep_2023 = df_registrations[df_registrations['submission_time'].dt.year == 2023]
df_registrations_sep_2023 = df_registrations_sep_2023[df_registrations_sep_2023['submission_time'].dt.month == 9]

# Calculate distinct count of main contacts from df_registrations
total_registrations = df_registrations_sep_2023['main_contact'].nunique()

# Define a custom function to calculate total calls made
def total_calls_made_func(seriessep):
    return seriessep.loc[~seriessep.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED'])].replace('None', '0').astype(int).sum()

# Summarize registrations by country
summary_registrations_sept2023 = df_verifications_sep_2023.groupby('country').agg(
    sample_pasted=('main_contact', 'nunique'),  # Using 'nunique' directly as an aggregation function
    verified_registrations=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    partially_verified=('verified_cc', lambda x: x.isin(['PARTIALLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(df_verifications_sep_2023['verified_cc'] != 'NOT CONTACTED') & (df_verifications_sep_2023['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_calls_made=('call_attempts', total_calls_made_func),  # Using custom aggregation function
    customers_reached=('call_status', lambda x: x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT']).sum()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA',
                                                     'PARTIALLY VERIFIED', 
                                                     'PARTIALLY VERIFIED - DORMANT', 
                                                     'DORMANT']).sum())
).reset_index()


# Rename the count columns for clarity
summary_registrations_sept2023.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Calculate total registrations by country
total_registrations_by_country = df_registrations_sep_2023.groupby('country')['main_contact'].nunique().reset_index()
total_registrations_by_country.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Merge total registrations by country with summary_registrations
summary_registrations_sept2023 = pd.merge(summary_registrations_sept2023, total_registrations_by_country, on='country', how='left')

# Calculate verification rate
summary_registrations_sept2023['old_verification_rate'] = (summary_registrations_sept2023['verified_registrations'] / summary_registrations_sept2023['total_verified']) * 100

# Add "%" symbol to verification_rate column
summary_registrations_sept2023['old_verification_rate'] = summary_registrations_sept2023['old_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage correctly
summary_registrations_sept2023['sample_percentage'] = (summary_registrations_sept2023['sample_pasted'] / summary_registrations_sept2023['total_registrations']) * 100

# Add "%" symbol to sample_percentage column
summary_registrations_sept2023['sample_percentage'] = summary_registrations_sept2023['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call rate correctly
summary_registrations_sept2023['call_rate'] = (summary_registrations_sept2023['customer_called'] / summary_registrations_sept2023['sample_pasted']) * 100

# Add "%" symbol to call_rate column
summary_registrations_sept2023['call_rate'] = summary_registrations_sept2023['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate correctly
summary_registrations_sept2023['reachability_rate'] = (summary_registrations_sept2023['customers_reached'] / summary_registrations_sept2023['customer_called']) * 100

# Add "%" symbol to reachability_rate column
summary_registrations_sept2023['reachability_rate'] = summary_registrations_sept2023['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate correctly
summary_registrations_sept2023['percentage_fully_verified'] = (summary_registrations_sept2023['fully_verified'] / summary_registrations_sept2023['sample_pasted']) * 100

# Add "%" symbol to fully verified_rate column
summary_registrations_sept2023['percentage_fully_verified'] = summary_registrations_sept2023['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification rate correctly
summary_registrations_sept2023['new_verification_rate'] = (summary_registrations_sept2023['fully_verified'] / summary_registrations_sept2023['customer_called']) * 100

# Add "%" symbol to new_verification rate column
summary_registrations_sept2023['new_verification_rate'] = summary_registrations_sept2023['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Add a row to the summary DataFrame for total registrations across all countries
total_row = pd.DataFrame({'country': ['Total'],
                          'total_registrations': [summary_registrations_sept2023['total_registrations'].sum()],
                          'sample_pasted': [summary_registrations_sept2023['sample_pasted'].sum()],
                          'verified_registrations': [summary_registrations_sept2023['verified_registrations'].sum()],
                          'fully_verified': [summary_registrations_sept2023['fully_verified'].sum()],
                          'partially_verified': [summary_registrations_sept2023['partially_verified'].sum()],
                          'customer_called': [summary_registrations_sept2023['customer_called'].sum()],
                          'total_calls_made': [summary_registrations_sept2023['total_calls_made'].sum()],
                          'customers_reached': [summary_registrations_sept2023['customers_reached'].sum()],
                          'total_verified': [summary_registrations_sept2023['total_verified'].sum()]})

# Calculate total verification rate
old_verification_rate = (total_row['verified_registrations'] / total_row['total_verified']) * 100

# Add total verification rate to the total row
total_row['old_verification_rate'] = old_verification_rate.apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage for total row
total_row['sample_percentage'] = (total_row['sample_pasted'] / total_row['total_registrations']) * 100

# Add "%" symbol to sample percentage column in the total row
total_row['sample_percentage'] = total_row['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call_rate for total row
total_row['call_rate'] = (total_row['customer_called'] / total_row['sample_pasted']) * 100

# Add "%" symbol to call_rate column in the total row
total_row['call_rate'] = total_row['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate for total row
total_row['reachability_rate'] = (total_row['customers_reached'] / total_row['customer_called']) * 100

# Add "%" symbol to sample_pasted column in the total row
total_row['reachability_rate'] = total_row['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate for total row
total_row['percentage_fully_verified'] = (total_row['fully_verified'] / total_row['sample_pasted']) * 100

# Add "%" symbol to fully verified column in the total row
total_row['percentage_fully_verified'] = total_row['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification_rate for total row
total_row['new_verification_rate'] = (total_row['fully_verified'] / total_row['customer_called']) * 100

# Add "%" symbol to new_verification_rate column in the total row
total_row['new_verification_rate'] = total_row['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))


# Concatenate summary registrations with total row
summary_registrations_sept2023 = pd.concat([summary_registrations_sept2023, total_row], ignore_index=True)

# Selecting the desired columns
selected_columns = ['total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'old_verification_rate', 'new_verification_rate', 'customer_called','total_calls_made', 'call_rate', 'customers_reached', 'reachability_rate', 'fully_verified', 'partially_verified', 'percentage_fully_verified']


# Adding the 'country' column back as index
summary_registrations_sept2023.reset_index(inplace=True)

# Reordering the columns to include 'country' as the first column
summary_registrations_sept2023 = summary_registrations_sept2023[['country'] + selected_columns]

# Adding a title to the DataFrame
summary_registrations_sept2023_title = "September 2023 Registrations"
summary_registrations_sept2023 = pd.concat([pd.DataFrame([summary_registrations_sept2023_title], columns=['Title']), summary_registrations_sept2023],
                                         ignore_index=True)

# Print the summary
print(summary_registrations_sept2023)


                          Title        country  total_registrations  \
0  September 2023 Registrations            NaN                  NaN   
1                           NaN            DRC               6246.0   
2                           NaN          KENYA              11908.0   
3                           NaN     MOZAMBIQUE               4481.0   
4                           NaN        NIGERIA              16841.0   
5                           NaN     SOMALILAND               2203.0   
6                           NaN  SOUTH SOMALIA               2384.0   
7                           NaN       TANZANIA               6635.0   
8                           NaN          Total              50698.0   

   sample_pasted sample_percentage  verified_registrations  \
0            NaN               NaN                     NaN   
1         2737.0            43.82%                   986.0   
2         3961.0            33.26%                  2184.0   
3         2721.0            60.72%       

In [13]:
####OCTOBER REGISTRATIONS
# Filter DataFrame for submissions in October 2023
df_verifications_oct_2023 = df_verifications[df_verifications['submission_time'].dt.year == 2023]
df_verifications_oct_2023 = df_verifications_oct_2023[df_verifications_oct_2023['submission_time'].dt.month == 10]

# Filter DataFrame for registrations in October 2023
df_registrations_oct_2023 = df_registrations[df_registrations['submission_time'].dt.year == 2023]
df_registrations_oct_2023 = df_registrations_oct_2023[df_registrations_oct_2023['submission_time'].dt.month == 10]

# Calculate distinct count of main contacts from df_registrations
total_registrations = df_registrations_oct_2023['main_contact'].nunique()

# Define a custom function to calculate total calls made
def total_calls_made_func(seriesoct):
    return seriesoct.loc[~seriesoct.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED'])].replace('None', '0').astype(int).sum()

# Summarize registrations by country
summary_registrations_oct2023 = df_verifications_oct_2023.groupby('country').agg(
    sample_pasted=('main_contact', 'nunique'),  # Using 'nunique' directly as an aggregation function
    verified_registrations=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    partially_verified=('verified_cc', lambda x: x.isin(['PARTIALLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(df_verifications_oct_2023['verified_cc'] != 'NOT CONTACTED') & (df_verifications_oct_2023['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_calls_made=('call_attempts', total_calls_made_func),  # Using custom aggregation function
    customers_reached=('call_status', lambda x: x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT']).sum()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA',
                                                     'PARTIALLY VERIFIED', 
                                                     'PARTIALLY VERIFIED - DORMANT', 
                                                     'DORMANT']).sum())
).reset_index()


# Rename the count columns for clarity
summary_registrations_oct2023.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Calculate total registrations by country
total_registrations_by_country = df_registrations_oct_2023.groupby('country')['main_contact'].nunique().reset_index()
total_registrations_by_country.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Merge total registrations by country with summary_registrations
summary_registrations_oct2023 = pd.merge(summary_registrations_oct2023, total_registrations_by_country, on='country', how='left')

# Calculate verification rate
summary_registrations_oct2023['old_verification_rate'] = (summary_registrations_oct2023['verified_registrations'] / summary_registrations_oct2023['total_verified']) * 100

# Add "%" symbol to verification_rate column
summary_registrations_oct2023['old_verification_rate'] = summary_registrations_oct2023['old_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage correctly
summary_registrations_oct2023['sample_percentage'] = (summary_registrations_oct2023['sample_pasted'] / summary_registrations_oct2023['total_registrations']) * 100

# Add "%" symbol to sample_percentage column
summary_registrations_oct2023['sample_percentage'] = summary_registrations_oct2023['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call rate correctly
summary_registrations_oct2023['call_rate'] = (summary_registrations_oct2023['customer_called'] / summary_registrations_oct2023['sample_pasted']) * 100

# Add "%" symbol to call_rate column
summary_registrations_oct2023['call_rate'] = summary_registrations_oct2023['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate correctly
summary_registrations_oct2023['reachability_rate'] = (summary_registrations_oct2023['customers_reached'] / summary_registrations_oct2023['customer_called']) * 100

# Add "%" symbol to reachability_rate column
summary_registrations_oct2023['reachability_rate'] = summary_registrations_oct2023['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate correctly
summary_registrations_oct2023['percentage_fully_verified'] = (summary_registrations_oct2023['fully_verified'] / summary_registrations_oct2023['sample_pasted']) * 100

# Add "%" symbol to fully verified_rate column
summary_registrations_oct2023['percentage_fully_verified'] = summary_registrations_oct2023['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification rate correctly
summary_registrations_oct2023['new_verification_rate'] = (summary_registrations_oct2023['fully_verified'] / summary_registrations_oct2023['customer_called']) * 100

# Add "%" symbol to new_verification rate column
summary_registrations_oct2023['new_verification_rate'] = summary_registrations_oct2023['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))


# Add a row to the summary DataFrame for total registrations across all countries
total_row = pd.DataFrame({'country': ['Total'],
                          'total_registrations': [summary_registrations_oct2023['total_registrations'].sum()],
                          'sample_pasted': [summary_registrations_oct2023['sample_pasted'].sum()],
                          'verified_registrations': [summary_registrations_oct2023['verified_registrations'].sum()],
                          'fully_verified': [summary_registrations_oct2023['fully_verified'].sum()],
                          'partially_verified': [summary_registrations_oct2023['partially_verified'].sum()],
                          'customer_called': [summary_registrations_oct2023['customer_called'].sum()],
                          'total_calls_made': [summary_registrations_oct2023['total_calls_made'].sum()],
                          'customers_reached': [summary_registrations_oct2023['customers_reached'].sum()],
                          'total_verified': [summary_registrations_oct2023['total_verified'].sum()]})

# Calculate total verification rate
old_verification_rate = (total_row['verified_registrations'] / total_row['total_verified']) * 100

# Add total verification rate to the total row
total_row['verification_rate'] = old_verification_rate.apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage for total row
total_row['sample_percentage'] = (total_row['sample_pasted'] / total_row['total_registrations']) * 100

# Add "%" symbol to sample percentage column in the total row
total_row['sample_percentage'] = total_row['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call_rate for total row
total_row['call_rate'] = (total_row['customer_called'] / total_row['sample_pasted']) * 100

# Add "%" symbol to call_rate column in the total row
total_row['call_rate'] = total_row['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate for total row
total_row['reachability_rate'] = (total_row['customers_reached'] / total_row['customer_called']) * 100

# Add "%" symbol to sample_pasted column in the total row
total_row['reachability_rate'] = total_row['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate for total row
total_row['percentage_fully_verified'] = (total_row['fully_verified'] / total_row['sample_pasted']) * 100

# Add "%" symbol to fully verified column in the total row
total_row['percentage_fully_verified'] = total_row['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))


# Calculate new_verification_rate for total row
total_row['new_verification_rate'] = (total_row['fully_verified'] / total_row['customer_called']) * 100

# Add "%" symbol to new_verification_rate column in the total row
total_row['new_verification_rate'] = total_row['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Concatenate summary registrations with total row
summary_registrations_oct2023 = pd.concat([summary_registrations_oct2023, total_row], ignore_index=True)

# Selecting the desired columns
selected_columns = ['total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'old_verification_rate', 'new_verification_rate', 'customer_called','total_calls_made', 'call_rate', 'customers_reached', 'reachability_rate', 'fully_verified', 'partially_verified', 'percentage_fully_verified']


# Adding the 'country' column back as index
summary_registrations_oct2023.reset_index(inplace=True)

# Reordering the columns to include 'country' as the first column
summary_registrations_oct2023 = summary_registrations_oct2023[['country'] + selected_columns]

# Adding a title to the DataFrame
summary_registrations_oct2023_title = "October 2023 Registrations"
summary_registrations_oct2023 = pd.concat([pd.DataFrame([summary_registrations_oct2023_title], columns=['Title']), summary_registrations_oct2023],
                                         ignore_index=True)


# Print the summary
print(summary_registrations_oct2023)


                        Title        country  total_registrations  \
0  October 2023 Registrations            NaN                  NaN   
1                         NaN            DRC               8960.0   
2                         NaN          KENYA              13928.0   
3                         NaN     MOZAMBIQUE               3370.0   
4                         NaN        NIGERIA              17406.0   
5                         NaN     SOMALILAND               4817.0   
6                         NaN  SOUTH SOMALIA               2942.0   
7                         NaN       TANZANIA               4532.0   
8                         NaN          Total              55955.0   

   sample_pasted sample_percentage  verified_registrations  \
0            NaN               NaN                     NaN   
1         3759.0            41.95%                  1025.0   
2         3596.0            25.82%                  1901.0   
3         1939.0            57.54%                  1223.0   

In [14]:
###NOVEMBER 2023 REGS
# Filter DataFrame for submissions in November 2023
df_verifications_nov_2023 = df_verifications[df_verifications['submission_time'].dt.year == 2023]
df_verifications_nov_2023 = df_verifications_nov_2023[df_verifications_nov_2023['submission_time'].dt.month == 11]

# Filter DataFrame for registrations in November 2023
df_registrations_nov_2023 = df_registrations[df_registrations['submission_time'].dt.year == 2023]
df_registrations_nov_2023 = df_registrations_nov_2023[df_registrations_nov_2023['submission_time'].dt.month == 11]

# Calculate distinct count of main contacts from df_registrations
total_registrations = df_registrations_nov_2023['main_contact'].nunique()

# Define a custom function to calculate total calls made
def total_calls_made_func(seriesnov):
    return seriesnov.loc[~seriesnov.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED'])].replace('None', '0').astype(int).sum()

# Summarize registrations by country
summary_registrations_nov2023 = df_verifications_nov_2023.groupby('country').agg(
    sample_pasted=('main_contact', 'nunique'),  # Using 'nunique' directly as an aggregation function
    verified_registrations=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    partially_verified=('verified_cc', lambda x: x.isin(['PARTIALLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(df_verifications_nov_2023['verified_cc'] != 'NOT CONTACTED') & (df_verifications_nov_2023['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_calls_made=('call_attempts', total_calls_made_func),  # Using custom aggregation function
    customers_reached=('call_status', lambda x: x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT']).sum()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA',
                                                     'PARTIALLY VERIFIED', 
                                                     'PARTIALLY VERIFIED - DORMANT', 
                                                     'DORMANT']).sum())
).reset_index()

# Rename the count columns for clarity
summary_registrations_nov2023.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Calculate total registrations by country
total_registrations_by_country = df_registrations_nov_2023.groupby('country')['main_contact'].nunique().reset_index()
total_registrations_by_country.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Merge total registrations by country with summary_registrations
summary_registrations_nov2023 = pd.merge(summary_registrations_nov2023, total_registrations_by_country, on='country', how='left')

# Calculate verification rate
summary_registrations_nov2023['old_verification_rate'] = (summary_registrations_nov2023['verified_registrations'] / summary_registrations_nov2023['total_verified']) * 100

# Add "%" symbol to verification_rate column
summary_registrations_nov2023['old_verification_rate'] = summary_registrations_nov2023['old_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage correctly
summary_registrations_nov2023['sample_percentage'] = (summary_registrations_nov2023['sample_pasted'] / summary_registrations_nov2023['total_registrations']) * 100

# Add "%" symbol to sample_percentage column
summary_registrations_nov2023['sample_percentage'] = summary_registrations_nov2023['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call rate correctly
summary_registrations_nov2023['call_rate'] = (summary_registrations_nov2023['customer_called'] / summary_registrations_nov2023['sample_pasted']) * 100

# Add "%" symbol to call_rate column
summary_registrations_nov2023['call_rate'] = summary_registrations_nov2023['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate correctly
summary_registrations_nov2023['reachability_rate'] = (summary_registrations_nov2023['customers_reached'] / summary_registrations_nov2023['customer_called']) * 100

# Add "%" symbol to reachability_rate column
summary_registrations_nov2023['reachability_rate'] = summary_registrations_nov2023['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate correctly
summary_registrations_nov2023['percentage_fully_verified'] = (summary_registrations_nov2023['fully_verified'] / summary_registrations_nov2023['sample_pasted']) * 100

# Add "%" symbol to fully verified_rate column
summary_registrations_nov2023['percentage_fully_verified'] = summary_registrations_nov2023['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification rate correctly
summary_registrations_nov2023['new_verification_rate'] = (summary_registrations_nov2023['fully_verified'] / summary_registrations_nov2023['customer_called']) * 100

# Add "%" symbol to new_verification rate column
summary_registrations_nov2023['new_verification_rate'] = summary_registrations_nov2023['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))



# Add a row to the summary DataFrame for total registrations across all countries
total_row = pd.DataFrame({'country': ['Total'],
                          'total_registrations': [summary_registrations_nov2023['total_registrations'].sum()],
                          'sample_pasted': [summary_registrations_nov2023['sample_pasted'].sum()],
                          'verified_registrations': [summary_registrations_nov2023['verified_registrations'].sum()],
                          'fully_verified': [summary_registrations_nov2023['fully_verified'].sum()],
                          'partially_verified': [summary_registrations_nov2023['partially_verified'].sum()],
                          'customer_called': [summary_registrations_nov2023['customer_called'].sum()],
                          'total_calls_made': [summary_registrations_nov2023['total_calls_made'].sum()],
                          'customers_reached': [summary_registrations_nov2023['customers_reached'].sum()],
                          'total_verified': [summary_registrations_nov2023['total_verified'].sum()]})

# Calculate total verification rate
old_verification_rate = (total_row['verified_registrations'] / total_row['total_verified']) * 100

# Add total verification rate to the total row
total_row['old_verification_rate'] = old_verification_rate.apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage for total row
total_row['sample_percentage'] = (total_row['sample_pasted'] / total_row['total_registrations']) * 100

# Add "%" symbol to sample percentage column in the total row
total_row['sample_percentage'] = total_row['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call_rate for total row
total_row['call_rate'] = (total_row['customer_called'] / total_row['sample_pasted']) * 100

# Add "%" symbol to call_rate column in the total row
total_row['call_rate'] = total_row['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate for total row
total_row['reachability_rate'] = (total_row['customers_reached'] / total_row['customer_called']) * 100

# Add "%" symbol to sample_pasted column in the total row
total_row['reachability_rate'] = total_row['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate for total row
total_row['percentage_fully_verified'] = (total_row['fully_verified'] / total_row['sample_pasted']) * 100

# Add "%" symbol to fully verified column in the total row
total_row['percentage_fully_verified'] = total_row['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification_rate for total row
total_row['new_verification_rate'] = (total_row['fully_verified'] / total_row['customer_called']) * 100

# Add "%" symbol to new_verification_rate column in the total row
total_row['new_verification_rate'] = total_row['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))


# Concatenate summary registrations with total row
summary_registrations_nov2023 = pd.concat([summary_registrations_nov2023, total_row], ignore_index=True)

# Selecting the desired columns
selected_columns = ['total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'old_verification_rate', 'new_verification_rate', 'customer_called','total_calls_made', 'call_rate', 'customers_reached', 'reachability_rate', 'fully_verified', 'partially_verified', 'percentage_fully_verified']


# Adding the 'country' column back as index
summary_registrations_nov2023.reset_index(inplace=True)

# Reordering the columns to include 'country' as the first column
summary_registrations_nov2023 = summary_registrations_nov2023[['country'] + selected_columns]

# Adding a title to the DataFrame
summary_registrations_nov2023_title = "November 2023 Registrations"
summary_registrations_nov2023 = pd.concat([pd.DataFrame([summary_registrations_nov2023_title], columns=['Title']), summary_registrations_nov2023],
                                         ignore_index=True)

# Print the summary
print(summary_registrations_nov2023)


                         Title        country  total_registrations  \
0  November 2023 Registrations            NaN                  NaN   
1                          NaN            DRC               8337.0   
2                          NaN          KENYA              14097.0   
3                          NaN     MOZAMBIQUE               4987.0   
4                          NaN        NIGERIA              23221.0   
5                          NaN     SOMALILAND               1145.0   
6                          NaN  SOUTH SOMALIA                582.0   
7                          NaN       TANZANIA               3215.0   
8                          NaN          Total              55584.0   

   sample_pasted sample_percentage  verified_registrations  \
0            NaN               NaN                     NaN   
1         4334.0            51.99%                  1236.0   
2         5008.0            35.53%                  2423.0   
3         2975.0            59.66%                 

In [15]:
###DECEMBER 2023 REGS
# Filter DataFrame for submissions in December 2023
df_verifications_dec_2023 = df_verifications[df_verifications['submission_time'].dt.year == 2023]
df_verifications_dec_2023 = df_verifications_dec_2023[df_verifications_dec_2023['submission_time'].dt.month == 12]

# Filter DataFrame for registrations in December 2023
df_registrations_dec_2023 = df_registrations[df_registrations['submission_time'].dt.year == 2023]
df_registrations_dec_2023 = df_registrations_dec_2023[df_registrations_dec_2023['submission_time'].dt.month == 12]

# Calculate distinct count of main contacts from df_registrations
total_registrations = df_registrations_dec_2023['main_contact'].nunique()

# Define a custom function to calculate total calls made
def total_calls_made_func(seriesdec):
    return seriesdec.loc[~seriesdec.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED'])].replace('None', '0').astype(int).sum()

# Summarize registrations by country
summary_registrations_dec2023 = df_verifications_dec_2023.groupby('country').agg(
    sample_pasted=('main_contact', 'nunique'),  # Using 'nunique' directly as an aggregation function
    verified_registrations=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    partially_verified=('verified_cc', lambda x: x.isin(['PARTIALLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(df_verifications_dec_2023['verified_cc'] != 'NOT CONTACTED') & (df_verifications_dec_2023['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_calls_made=('call_attempts', total_calls_made_func),  # Using custom aggregation function
    customers_reached=('call_status', lambda x: x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT']).sum()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA',
                                                     'PARTIALLY VERIFIED', 
                                                     'PARTIALLY VERIFIED - DORMANT', 
                                                     'DORMANT']).sum())
).reset_index()

# Rename the count columns for clarity
summary_registrations_dec2023.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Calculate total registrations by country
total_registrations_by_country = df_registrations_dec_2023.groupby('country')['main_contact'].nunique().reset_index()
total_registrations_by_country.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Merge total registrations by country with summary_registrations
summary_registrations_dec2023 = pd.merge(summary_registrations_dec2023, total_registrations_by_country, on='country', how='left')

# Calculate verification rate
summary_registrations_dec2023['old_verification_rate'] = (summary_registrations_dec2023['verified_registrations'] / summary_registrations_dec2023['total_verified']) * 100

# Add "%" symbol to verification_rate column
summary_registrations_dec2023['old_verification_rate'] = summary_registrations_dec2023['old_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage correctly
summary_registrations_dec2023['sample_percentage'] = (summary_registrations_dec2023['sample_pasted'] / summary_registrations_dec2023['total_registrations']) * 100

# Add "%" symbol to sample_percentage column
summary_registrations_dec2023['sample_percentage'] = summary_registrations_dec2023['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call rate correctly
summary_registrations_dec2023['call_rate'] = (summary_registrations_dec2023['customer_called'] / summary_registrations_dec2023['sample_pasted']) * 100

# Add "%" symbol to call_rate column
summary_registrations_dec2023['call_rate'] = summary_registrations_dec2023['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate correctly
summary_registrations_dec2023['reachability_rate'] = (summary_registrations_dec2023['customers_reached'] / summary_registrations_dec2023['customer_called']) * 100

# Add "%" symbol to reachability_rate column
summary_registrations_dec2023['reachability_rate'] = summary_registrations_dec2023['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate correctly
summary_registrations_dec2023['percentage_fully_verified'] = (summary_registrations_dec2023['fully_verified'] / summary_registrations_dec2023['sample_pasted']) * 100

# Add "%" symbol to fully verified_rate column
summary_registrations_dec2023['percentage_fully_verified'] = summary_registrations_dec2023['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification rate correctly
summary_registrations_dec2023['new_verification_rate'] = (summary_registrations_dec2023['fully_verified'] / summary_registrations_dec2023['customer_called']) * 100

# Add "%" symbol to new_verification rate column
summary_registrations_dec2023['new_verification_rate'] = summary_registrations_dec2023['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))


# Add a row to the summary DataFrame for total registrations across all countries
total_row = pd.DataFrame({'country': ['Total'],
                          'total_registrations': [summary_registrations_dec2023['total_registrations'].sum()],
                          'sample_pasted': [summary_registrations_dec2023['sample_pasted'].sum()],
                          'verified_registrations': [summary_registrations_dec2023['verified_registrations'].sum()],
                          'fully_verified': [summary_registrations_dec2023['fully_verified'].sum()],
                          'partially_verified': [summary_registrations_dec2023['partially_verified'].sum()],
                          'customer_called': [summary_registrations_dec2023['customer_called'].sum()],
                          'total_calls_made': [summary_registrations_dec2023['total_calls_made'].sum()],
                          'customers_reached': [summary_registrations_dec2023['customers_reached'].sum()],
                          'total_verified': [summary_registrations_dec2023['total_verified'].sum()]})

# Calculate total verification rate
old_verification_rate = (total_row['verified_registrations'] / total_row['total_verified']) * 100

# Add total verification rate to the total row
total_row['old_verification_rate'] = old_verification_rate.apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage for total row
total_row['sample_percentage'] = (total_row['sample_pasted'] / total_row['total_registrations']) * 100

# Add "%" symbol to sample percentage column in the total row
total_row['sample_percentage'] = total_row['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call_rate for total row
total_row['call_rate'] = (total_row['customer_called'] / total_row['sample_pasted']) * 100

# Add "%" symbol to call_rate column in the total row
total_row['call_rate'] = total_row['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate for total row
total_row['reachability_rate'] = (total_row['customers_reached'] / total_row['customer_called']) * 100

# Add "%" symbol to sample_pasted column in the total row
total_row['reachability_rate'] = total_row['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate for total row
total_row['percentage_fully_verified'] = (total_row['fully_verified'] / total_row['sample_pasted']) * 100

# Add "%" symbol to fully verified column in the total row
total_row['percentage_fully_verified'] = total_row['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Concatenate summary registrations with total row
summary_registrations_dec2023 = pd.concat([summary_registrations_dec2023, total_row], ignore_index=True)

# Calculate new_verification_rate for total row
total_row['new_verification_rate'] = (total_row['fully_verified'] / total_row['customer_called']) * 100

# Add "%" symbol to new_verification_rate column in the total row
total_row['new_verification_rate'] = total_row['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Selecting the desired columns
selected_columns = ['total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'old_verification_rate', 'new_verification_rate', 'customer_called','total_calls_made', 'call_rate', 'customers_reached', 'reachability_rate', 'fully_verified', 'partially_verified', 'percentage_fully_verified']


# Adding the 'country' column back as index
summary_registrations_dec2023.reset_index(inplace=True)

# Reordering the columns to include 'country' as the first column
summary_registrations_dec2023 = summary_registrations_dec2023[['country'] + selected_columns]

# Adding a title to the DataFrame
summary_registrations_dec2023_title = "December 2023 Registrations"
summary_registrations_dec2023 = pd.concat([pd.DataFrame([summary_registrations_dec2023_title], columns=['Title']), summary_registrations_dec2023],
                                         ignore_index=True)
# Print the summary
print(summary_registrations_dec2023)


                         Title        country  total_registrations  \
0  December 2023 Registrations            NaN                  NaN   
1                          NaN            DRC               5134.0   
2                          NaN          KENYA               5934.0   
3                          NaN     MOZAMBIQUE               3827.0   
4                          NaN        NIGERIA              18828.0   
5                          NaN     SOMALILAND               2339.0   
6                          NaN  SOUTH SOMALIA               3072.0   
7                          NaN       TANZANIA               2153.0   
8                          NaN          Total              41287.0   

   sample_pasted sample_percentage  verified_registrations  \
0            NaN               NaN                     NaN   
1         2396.0            46.67%                   541.0   
2         3202.0            53.96%                  1279.0   
3         2392.0            62.50%                 

In [16]:
# Fill 'None' values with 0 in the relevant columns
#df_verifications['call_attempts'] = df_verifications['call_attempts'].fillna(0)

#df_registrations['call_attempts'] = df_registrations['call_attempts'].fillna(0)


In [17]:
# Filter DataFrame for submissions in January 2024

df_verifications_jan_2024 = df_verifications[df_verifications['submission_time'].dt.year == 2024]
df_verifications_jan_2024 = df_verifications_jan_2024[df_verifications_jan_2024['submission_time'].dt.month == 1]

# Filter DataFrame for registrations in January 2024
df_registrations_jan_2024 = df_registrations[df_registrations['submission_time'].dt.year == 2024]
df_registrations_jan_2024 = df_registrations_jan_2024[df_registrations_jan_2024['submission_time'].dt.month == 1]

# Calculate distinct count of main contacts from df_registrations
total_registrations = df_registrations_jan_2024['main_contact'].nunique()

# Define a custom function to calculate total calls made
def total_calls_made_func(seriesjan24):
    return seriesjan24.loc[~seriesjan24.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED'])].replace('None', '0').astype(int).sum()

# Summarize registrations by country
summary_registrations_jan2024 = df_verifications_jan_2024.groupby('country').agg(
    sample_pasted=('main_contact', 'nunique'),  # Using 'nunique' directly as an aggregation function
    verified_registrations=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    partially_verified=('verified_cc', lambda x: x.isin(['PARTIALLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(df_verifications_jan_2024['verified_cc'] != 'NOT CONTACTED') & (df_verifications_jan_2024['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_calls_made=('call_attempts', total_calls_made_func),  # Using custom aggregation function
    customers_reached=('call_status', lambda x: x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT']).sum()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA',
                                                     'PARTIALLY VERIFIED', 
                                                     'PARTIALLY VERIFIED - DORMANT', 
                                                     'DORMANT']).sum())
).reset_index()
# Rename the count columns for clarity
summary_registrations_jan2024.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Calculate total registrations by country
total_registrations_by_country = df_registrations_jan_2024.groupby('country')['main_contact'].nunique().reset_index()
total_registrations_by_country.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Merge total registrations by country with summary_registrations
summary_registrations_jan2024 = pd.merge(summary_registrations_jan2024, total_registrations_by_country, on='country', how='left')

# Calculate verification rate
summary_registrations_jan2024['old_verification_rate'] = (summary_registrations_jan2024['verified_registrations'] / summary_registrations_jan2024['total_verified']) * 100

# Add "%" symbol to verification_rate column
summary_registrations_jan2024['old_verification_rate'] = summary_registrations_jan2024['old_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage correctly
summary_registrations_jan2024['sample_percentage'] = (summary_registrations_jan2024['sample_pasted'] / summary_registrations_jan2024['total_registrations']) * 100

# Add "%" symbol to sample_percentage column
summary_registrations_jan2024['sample_percentage'] = summary_registrations_jan2024['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call rate correctly
summary_registrations_jan2024['call_rate'] = (summary_registrations_jan2024['customer_called'] / summary_registrations_jan2024['sample_pasted']) * 100

# Add "%" symbol to call_rate column
summary_registrations_jan2024['call_rate'] = summary_registrations_jan2024['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate correctly
summary_registrations_jan2024['reachability_rate'] = (summary_registrations_jan2024['customers_reached'] / summary_registrations_jan2024['customer_called']) * 100

# Add "%" symbol to reachability_rate column
summary_registrations_jan2024['reachability_rate'] = summary_registrations_jan2024['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate correctly
summary_registrations_jan2024['percentage_fully_verified'] = (summary_registrations_jan2024['fully_verified'] / summary_registrations_jan2024['sample_pasted']) * 100

# Add "%" symbol to fully verified_rate column
summary_registrations_jan2024['percentage_fully_verified'] = summary_registrations_jan2024['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification rate correctly
summary_registrations_jan2024['new_verification_rate'] = (summary_registrations_jan2024['fully_verified'] / summary_registrations_jan2024['customer_called']) * 100

# Add "%" symbol to new_verification rate column
summary_registrations_jan2024['new_verification_rate'] = summary_registrations_jan2024['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))


# Add a row to the summary DataFrame for total registrations across all countries
total_row = pd.DataFrame({'country': ['Total'],
                          'total_registrations': [summary_registrations_jan2024['total_registrations'].sum()],
                          'sample_pasted': [summary_registrations_jan2024['sample_pasted'].sum()],
                          'verified_registrations': [summary_registrations_jan2024['verified_registrations'].sum()],
                          'fully_verified': [summary_registrations_jan2024['fully_verified'].sum()],
                          'partially_verified': [summary_registrations_jan2024['partially_verified'].sum()],
                          'customer_called': [summary_registrations_jan2024['customer_called'].sum()],
                          'total_calls_made': [summary_registrations_jan2024['total_calls_made'].sum()],
                          'customers_reached': [summary_registrations_jan2024['customers_reached'].sum()],
                          'total_verified': [summary_registrations_jan2024['total_verified'].sum()]})

# Calculate total verification rate
old_verification_rate = (total_row['verified_registrations'] / total_row['total_verified']) * 100

# Add total verification rate to the total row
total_row['old_verification_rate'] = old_verification_rate.apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage for total row
total_row['sample_percentage'] = (total_row['sample_pasted'] / total_row['total_registrations']) * 100

# Add "%" symbol to sample percentage column in the total row
total_row['sample_percentage'] = total_row['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call_rate for total row
total_row['call_rate'] = (total_row['customer_called'] / total_row['sample_pasted']) * 100

# Add "%" symbol to call_rate column in the total row
total_row['call_rate'] = total_row['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate for total row
total_row['reachability_rate'] = (total_row['customers_reached'] / total_row['customer_called']) * 100

# Add "%" symbol to sample_pasted column in the total row
total_row['reachability_rate'] = total_row['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate for total row
total_row['percentage_fully_verified'] = (total_row['fully_verified'] / total_row['sample_pasted']) * 100

# Add "%" symbol to fully verified column in the total row
total_row['percentage_fully_verified'] = total_row['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification_rate for total row
total_row['new_verification_rate'] = (total_row['fully_verified'] / total_row['customer_called']) * 100

# Add "%" symbol to new_verification_rate column in the total row
total_row['new_verification_rate'] = total_row['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Concatenate summary registrations with total row
summary_registrations_jan2024 = pd.concat([summary_registrations_jan2024, total_row], ignore_index=True)

# Selecting the desired columns
selected_columns = ['total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'old_verification_rate', 'new_verification_rate', 'customer_called','total_calls_made', 'call_rate', 'customers_reached', 'reachability_rate', 'fully_verified', 'partially_verified', 'percentage_fully_verified']


# Adding the 'country' column back as index
summary_registrations_jan2024.reset_index(inplace=True)

# Reordering the columns to include 'country' as the first column
summary_registrations_jan2024 = summary_registrations_jan2024[['country'] + selected_columns]

# Adding a title to the DataFrame
summary_registrations_jan2024_title = "January 2024 Registrations"
summary_registrations_jan2024 = pd.concat([pd.DataFrame([summary_registrations_jan2024_title], columns=['Title']), summary_registrations_jan2024],
                                         ignore_index=True)

# Print the summary
print(summary_registrations_jan2024)


                        Title        country  total_registrations  \
0  January 2024 Registrations            NaN                  NaN   
1                         NaN            DRC               5222.0   
2                         NaN          KENYA               6243.0   
3                         NaN     MOZAMBIQUE               3360.0   
4                         NaN        NIGERIA              19050.0   
5                         NaN     SOMALILAND                 62.0   
6                         NaN  SOUTH SOMALIA                210.0   
7                         NaN       TANZANIA               3763.0   
8                         NaN          Total              37910.0   

   sample_pasted sample_percentage  verified_registrations  \
0            NaN               NaN                     NaN   
1         1470.0            28.15%                   502.0   
2         3470.0            55.58%                  1890.0   
3         2290.0            68.15%                  1609.0   

In [18]:
# Filter DataFrame for submissions in February 2024
df_verifications_feb_2024 = df_verifications[df_verifications['submission_time'].dt.year == 2024]
df_verifications_feb_2024 = df_verifications_feb_2024[df_verifications_feb_2024['submission_time'].dt.month == 2]

# Filter DataFrame for registrations in February 2024
df_registrations_feb_2024 = df_registrations[df_registrations['submission_time'].dt.year == 2024]
df_registrations_feb_2024 = df_registrations_feb_2024[df_registrations_feb_2024['submission_time'].dt.month == 2]

# Calculate distinct count of main contacts from df_registrations
total_registrations = df_registrations_feb_2024['main_contact'].nunique()


# Define a custom function to calculate total calls made
def total_calls_made_func(seriesfeb24):
    return seriesfeb24.loc[~seriesfeb24.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED'])].replace('None', '0').astype(int).sum()

# Summarize registrations by country
summary_registrations_feb2024 = df_verifications_feb_2024.groupby('country').agg(
    sample_pasted=('main_contact', 'nunique'),  # Using 'nunique' directly as an aggregation function
    verified_registrations=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    partially_verified=('verified_cc', lambda x: x.isin(['PARTIALLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(df_verifications_feb_2024['verified_cc'] != 'NOT CONTACTED') & (df_verifications_feb_2024['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_calls_made=('call_attempts', total_calls_made_func),  # Using custom aggregation function
    customers_reached=('call_status', lambda x: x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT']).sum()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA',
                                                     'PARTIALLY VERIFIED', 
                                                     'PARTIALLY VERIFIED - DORMANT', 
                                                     'DORMANT']).sum())
).reset_index()
# Rename the count columns for clarity
summary_registrations_feb2024.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Calculate total registrations by country
total_registrations_by_country = df_registrations_feb_2024.groupby('country')['main_contact'].nunique().reset_index()
total_registrations_by_country.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Merge total registrations by country with summary_registrations
summary_registrations_feb2024 = pd.merge(summary_registrations_feb2024, total_registrations_by_country, on='country', how='left')

# Calculate verification rate
summary_registrations_feb2024['old_verification_rate'] = (summary_registrations_feb2024['verified_registrations'] / summary_registrations_feb2024['total_verified']) * 100

# Add "%" symbol to verification_rate column
summary_registrations_feb2024['old_verification_rate'] = summary_registrations_feb2024['old_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage correctly
summary_registrations_feb2024['sample_percentage'] = (summary_registrations_feb2024['sample_pasted'] / summary_registrations_feb2024['total_registrations']) * 100

# Add "%" symbol to sample_percentage column
summary_registrations_feb2024['sample_percentage'] = summary_registrations_feb2024['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call rate correctly
summary_registrations_feb2024['call_rate'] = (summary_registrations_feb2024['customer_called'] / summary_registrations_feb2024['sample_pasted']) * 100

# Add "%" symbol to call_rate column
summary_registrations_feb2024['call_rate'] = summary_registrations_feb2024['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate correctly
summary_registrations_feb2024['reachability_rate'] = (summary_registrations_feb2024['customers_reached'] / summary_registrations_feb2024['customer_called']) * 100

# Add "%" symbol to reachability_rate column
summary_registrations_feb2024['reachability_rate'] = summary_registrations_feb2024['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate correctly
summary_registrations_feb2024['percentage_fully_verified'] = (summary_registrations_feb2024['fully_verified'] / summary_registrations_feb2024['sample_pasted']) * 100

# Add "%" symbol to fully verified_rate column
summary_registrations_feb2024['percentage_fully_verified'] = summary_registrations_feb2024['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification rate correctly
summary_registrations_feb2024['new_verification_rate'] = (summary_registrations_feb2024['fully_verified'] / summary_registrations_feb2024['customer_called']) * 100

# Add "%" symbol to new_verification rate column
summary_registrations_feb2024['new_verification_rate'] = summary_registrations_feb2024['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))


# Add a row to the summary DataFrame for total registrations across all countries
total_row = pd.DataFrame({'country': ['Total'],
                          'total_registrations': [summary_registrations_feb2024['total_registrations'].sum()],
                          'sample_pasted': [summary_registrations_feb2024['sample_pasted'].sum()],
                          'verified_registrations': [summary_registrations_feb2024['verified_registrations'].sum()],
                          'fully_verified': [summary_registrations_feb2024['fully_verified'].sum()],
                          'partially_verified': [summary_registrations_feb2024['partially_verified'].sum()],
                          'customer_called': [summary_registrations_feb2024['customer_called'].sum()],
                          'total_calls_made': [summary_registrations_feb2024['total_calls_made'].sum()],
                          'customers_reached': [summary_registrations_feb2024['customers_reached'].sum()],
                          'total_verified': [summary_registrations_feb2024['total_verified'].sum()]})

# Calculate total verification rate
old_verification_rate = (total_row['verified_registrations'] / total_row['total_verified']) * 100

# Add total verification rate to the total row
total_row['old_verification_rate'] = old_verification_rate.apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage for total row
total_row['sample_percentage'] = (total_row['sample_pasted'] / total_row['total_registrations']) * 100

# Add "%" symbol to sample percentage column in the total row
total_row['sample_percentage'] = total_row['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call_rate for total row
total_row['call_rate'] = (total_row['customer_called'] / total_row['sample_pasted']) * 100

# Add "%" symbol to call_rate column in the total row
total_row['call_rate'] = total_row['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate for total row
total_row['reachability_rate'] = (total_row['customers_reached'] / total_row['customer_called']) * 100

# Add "%" symbol to reachability_rate column in the total row
total_row['reachability_rate'] = total_row['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate for total row
total_row['percentage_fully_verified'] = (total_row['fully_verified'] / total_row['sample_pasted']) * 100

# Add "%" symbol to fully verified column in the total row
total_row['percentage_fully_verified'] = total_row['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification_rate for total row
total_row['new_verification_rate'] = (total_row['fully_verified'] / total_row['customer_called']) * 100

# Add "%" symbol to new_verification_rate column in the total row
total_row['new_verification_rate'] = total_row['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Concatenate summary registrations with total row
summary_registrations_feb2024 = pd.concat([summary_registrations_feb2024, total_row], ignore_index=True)

# Selecting the desired columns
selected_columns = ['total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'old_verification_rate', 'new_verification_rate', 'customer_called','total_calls_made', 'call_rate', 'customers_reached', 'reachability_rate', 'fully_verified', 'partially_verified', 'percentage_fully_verified']

# Adding the 'country' column back as index
summary_registrations_feb2024.reset_index(inplace=True)

# Reordering the columns to include 'country' as the first column
summary_registrations_feb2024 = summary_registrations_feb2024[['country'] + selected_columns]

# Adding a title to the DataFrame
summary_registrations_feb2024_title = "February 2024 Registrations"
summary_registrations_feb2024 = pd.concat([pd.DataFrame([summary_registrations_feb2024_title], columns=['Title']), summary_registrations_feb2024],
                                         ignore_index=True)

# Print the summary
print(summary_registrations_feb2024)



                         Title        country  total_registrations  \
0  February 2024 Registrations            NaN                  NaN   
1                          NaN            DRC              10124.0   
2                          NaN          KENYA               6055.0   
3                          NaN     MOZAMBIQUE               7258.0   
4                          NaN        NIGERIA              18963.0   
5                          NaN  SOUTH SOMALIA               1106.0   
6                          NaN       TANZANIA               4834.0   
7                          NaN          Total              48340.0   

   sample_pasted sample_percentage  verified_registrations  \
0            NaN               NaN                     NaN   
1         6630.0            65.49%                  2563.0   
2         3568.0            58.93%                  2221.0   
3         4425.0            60.97%                  2541.0   
4         6083.0            32.08%                  5280.0 

In [19]:
# Filter DataFrame for submissions in March 2024
df_verifications_mar_2024 = df_verifications[df_verifications['submission_time'].dt.year == 2024]
df_verifications_mar_2024 = df_verifications_mar_2024[df_verifications_mar_2024['submission_time'].dt.month == 3]

# Filter DataFrame for registrations in March 2024
df_registrations_mar_2024 = df_registrations[df_registrations['submission_time'].dt.year == 2024]
df_registrations_mar_2024 = df_registrations_mar_2024[df_registrations_mar_2024['submission_time'].dt.month == 3]

# Calculate distinct count of main contacts from df_registrations
total_registrations = df_registrations_mar_2024['main_contact'].nunique()

# Define a custom function to calculate total calls made
def total_calls_made_func(seriesmar24):
    return seriesmar24.loc[~seriesmar24.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED'])].replace('None', '0').astype(int).sum()

# Summarize registrations by country
summary_registrations_mar2024 = df_verifications_mar_2024.groupby('country').agg(
    sample_pasted=('main_contact', 'nunique'),  # Using 'nunique' directly as an aggregation function
    verified_registrations=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    partially_verified=('verified_cc', lambda x: x.isin(['PARTIALLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(df_verifications_mar_2024['verified_cc'] != 'NOT CONTACTED') & (df_verifications_mar_2024['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_calls_made=('call_attempts', total_calls_made_func),  # Using custom aggregation function
    customers_reached=('call_status', lambda x: x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT']).sum()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA',
                                                     'PARTIALLY VERIFIED', 
                                                     'PARTIALLY VERIFIED - DORMANT', 
                                                     'DORMANT']).sum())
).reset_index()
# Rename the count columns for clarity
summary_registrations_oct2023.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Calculate total registrations by country
total_registrations_by_country = df_registrations_mar_2024.groupby('country')['main_contact'].nunique().reset_index()
total_registrations_by_country.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Merge total registrations by country with summary_registrations
summary_registrations_mar2024 = pd.merge(summary_registrations_mar2024, total_registrations_by_country, on='country', how='left')

# Calculate verification rate
summary_registrations_mar2024['old_verification_rate'] = (summary_registrations_mar2024['verified_registrations'] / summary_registrations_mar2024['total_verified']) * 100

# Add "%" symbol to verification_rate column
summary_registrations_mar2024['old_verification_rate'] = summary_registrations_mar2024['old_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage correctly
summary_registrations_mar2024['sample_percentage'] = (summary_registrations_mar2024['sample_pasted'] / summary_registrations_mar2024['total_registrations']) * 100

# Add "%" symbol to sample_percentage column
summary_registrations_mar2024['sample_percentage'] = summary_registrations_mar2024['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call rate correctly
summary_registrations_mar2024['call_rate'] = (summary_registrations_mar2024['customer_called'] / summary_registrations_mar2024['sample_pasted']) * 100

# Add "%" symbol to call_rate column
summary_registrations_mar2024['call_rate'] = summary_registrations_mar2024['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate correctly
summary_registrations_mar2024['reachability_rate'] = (summary_registrations_mar2024['customers_reached'] / summary_registrations_mar2024['customer_called']) * 100

# Add "%" symbol to reachability_rate column
summary_registrations_mar2024['reachability_rate'] = summary_registrations_mar2024['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate correctly
summary_registrations_mar2024['percentage_fully_verified'] = (summary_registrations_mar2024['fully_verified'] / summary_registrations_mar2024['sample_pasted']) * 100

# Add "%" symbol to fully verified_rate column
summary_registrations_mar2024['percentage_fully_verified'] = summary_registrations_mar2024['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification rate correctly
summary_registrations_mar2024['new_verification_rate'] = (summary_registrations_mar2024['fully_verified'] / summary_registrations_mar2024['customer_called']) * 100

# Add "%" symbol to new_verification rate column
summary_registrations_mar2024['new_verification_rate'] = summary_registrations_mar2024['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Add a row to the summary DataFrame for total registrations across all countries
total_row = pd.DataFrame({'country': ['Total'],
                          'total_registrations': [summary_registrations_mar2024['total_registrations'].sum()],
                          'sample_pasted': [summary_registrations_mar2024['sample_pasted'].sum()],
                          'verified_registrations': [summary_registrations_mar2024['verified_registrations'].sum()],
                          'fully_verified': [summary_registrations_mar2024['fully_verified'].sum()],
                          'partially_verified': [summary_registrations_mar2024['partially_verified'].sum()],
                          'customer_called': [summary_registrations_mar2024['customer_called'].sum()],
                          'total_calls_made': [summary_registrations_mar2024['total_calls_made'].sum()],
                          'customers_reached': [summary_registrations_mar2024['customers_reached'].sum()],
                          'total_verified': [summary_registrations_mar2024['total_verified'].sum()]})

# Calculate total verification rate
old_verification_rate = (total_row['verified_registrations'] / total_row['total_verified']) * 100

# Add total verification rate to the total row
total_row['old_verification_rate'] = old_verification_rate.apply(lambda x: '{:.2f}%'.format(x))

# Calculate sample percentage for total row
total_row['sample_percentage'] = (total_row['sample_pasted'] / total_row['total_registrations']) * 100

# Add "%" symbol to sample percentage column in the total row
total_row['sample_percentage'] = total_row['sample_percentage'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate call_rate for total row
total_row['call_rate'] = (total_row['customer_called'] / total_row['sample_pasted']) * 100

# Add "%" symbol to call_rate column in the total row
total_row['call_rate'] = total_row['call_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate reachability_rate for total row
total_row['reachability_rate'] = (total_row['customers_reached'] / total_row['customer_called']) * 100

# Add "%" symbol to reachability_rate column in the total row
total_row['reachability_rate'] = total_row['reachability_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate fully verified_rate for total row
total_row['percentage_fully_verified'] = (total_row['fully_verified'] / total_row['sample_pasted']) * 100

# Add "%" symbol to fully verified column in the total row
total_row['percentage_fully_verified'] = total_row['percentage_fully_verified'].apply(lambda x: '{:.2f}%'.format(x))

# Calculate new_verification_rate for total row
total_row['new_verification_rate'] = (total_row['fully_verified'] / total_row['customer_called']) * 100

# Add "%" symbol to new_verification_rate column in the total row
total_row['new_verification_rate'] = total_row['new_verification_rate'].apply(lambda x: '{:.2f}%'.format(x))

# Concatenate summary registrations with total row
summary_registrations_mar2024 = pd.concat([summary_registrations_mar2024, total_row], ignore_index=True)

# Selecting the desired columns
selected_columns = ['total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'old_verification_rate', 'new_verification_rate', 'customer_called','total_calls_made', 'call_rate', 'customers_reached', 'reachability_rate', 'fully_verified', 'partially_verified', 'percentage_fully_verified']

# Adding the 'country' column back as index
summary_registrations_mar2024.reset_index(inplace=True)

# Reordering the columns to include 'country' as the first column
summary_registrations_mar2024 = summary_registrations_mar2024[['country'] + selected_columns]

# Adding a title to the DataFrame
summary_registrations_mar2024_title = "March 2024 Registrations"
summary_registrations_mar2024 = pd.concat([pd.DataFrame([summary_registrations_mar2024_title], columns=['Title']), summary_registrations_mar2024],
                                         ignore_index=True)

# Print the summary
print(summary_registrations_mar2024)



                      Title     country  total_registrations  sample_pasted  \
0  March 2024 Registrations         NaN                  NaN            NaN   
1                       NaN         DRC               8574.0         4787.0   
2                       NaN       KENYA               8282.0         4138.0   
3                       NaN  MOZAMBIQUE               2821.0         2114.0   
4                       NaN     NIGERIA              18383.0         6246.0   
5                       NaN    TANZANIA               3568.0         2657.0   
6                       NaN       Total              41628.0        19942.0   

  sample_percentage  verified_registrations old_verification_rate  \
0               NaN                     NaN                   NaN   
1            55.83%                  1748.0                88.82%   
2            49.96%                  1808.0                86.67%   
3            74.94%                   961.0                98.36%   
4            33.98%   

In [20]:
data = {
    'Name': ['total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'total_verified',
             'old_verification_rate', 'new_verification_rate', 'customer_called', 'total_calls_made', 'call_rate', 
             'customers_reached', 'reachability_rate'],
    'Meaning': ['Total number of unique registrations', 
                'total registrations included in the verification sample or total registrations sampled.',
                'Percentage of sample pasted relative to the total number of unique registrations',
                'Total number of registrations with verified status',
                'total number of samples verified',
                'Percentage of verified registrations relative to total verified',
                'Percentage of fully verified registrations relative to total no of customers called',
                'Total number of unique main contacts not marked as "NOT CONTACTED"',
                'Total number of call attempts for verified registrations',
                'Percentage of customers called relative to sample pasted',
                'Total number of customers reached',
                'Percentage of customers reached relative to customers called'],
    'Formula': ['total number of unique households registered',
                'total number of unique households contacted',
                '(sample_pasted / total_registrations) * 100',
                'Count of "FULLY VERIFIED" or "PARTIALLY VERIFIED" entries in in the verification status of the sampled registrations',
                'Count of DECLINE, DORMANT (HARD TO RESOLVE), FULLY VERIFIED, DOUBTFUL VERIFIED, ARTIALLY VERIFIED, PARTIALLY VERIFIED - DORMANT, WRONG DATA, DORMANT in the verification status of the sampled registrations',
                '(verified_registrations / total_verified) * 100',
                '(fully_verified_registrations / customers_called) * 100',
                'Count of unique main contacts in df_verifications where "verified_cc" is not "NOT CONTACTED"',
                'Count of entries in "call_attempts" column where "verified_cc" is not "NOT CONTACTED"',
                '(customer_called / sample_pasted) * 100',
                'Count of DECLINE, FOLLOW UP, REACHABLE, FULLY VERIFIED, VERIFIED, NOT WILLING TO PARTICIPATE, WRONG NUMBER, PARTIALLY VERIFIED,PARTIALLY VERIFIED - DORMANT in "call_status" column with specific statuses in the verification status of the sampled registrations',
                '(customers_reached / customer_called) * 100']
}


# Create the DataFrame
metrics_df = pd.DataFrame(data)

# Display the DataFrame
print(metrics_df)


                      Name                                            Meaning  \
0      total_registrations               Total number of unique registrations   
1            sample_pasted  total registrations included in the verificati...   
2        sample_percentage  Percentage of sample pasted relative to the to...   
3   verified_registrations  Total number of registrations with verified st...   
4           total_verified                   total number of samples verified   
5    old_verification_rate  Percentage of verified registrations relative ...   
6    new_verification_rate  Percentage of fully verified registrations rel...   
7          customer_called  Total number of unique main contacts not marke...   
8         total_calls_made  Total number of call attempts for verified reg...   
9                call_rate  Percentage of customers called relative to sam...   
10       customers_reached                  Total number of customers reached   
11       reachability_rate  

In [21]:
import os
import pandas as pd
import numpy as np

# Set options
# options(scipen = 999)
np.set_printoptions(suppress=True)

if os.path.exists("D:/Burn Manufacturing/"):
    server_user_path = "D:/Burn Manufacturing/"
else:
    server_user_path = f"C:/Users/{os.getlogin()}/Burn Manufacturing/"

# Define functions
def to_date_excel(dt):
    return pd.to_datetime(dt, origin='1899-12-30')

def to_date_r(dt):
    return pd.to_datetime(dt, origin='1970-01-01')

def to_date_different_orders(dt):
    return pd.to_datetime(dt, errors='coerce')

def convert_to_date(dt):
    if pd.isna(dt):
        return pd.NaT
    elif '-' in dt:
        return pd.to_datetime(dt)
    else:
        return to_date_excel(dt)

def trim_upper(data):
    return data.apply(lambda x: x.str.strip().str.upper() if x.dtype == "object" else x)

def count_na_func(x):
    return sum(pd.isna(x) | (x == ""))

# Set email password
os.environ["SMTP_PASSWORD"] = "Burn@twentytwenty01"

# Set working directory
os.chdir("D:/Burn Manufacturing/CX Verification Calls - CX Verifications App/App Exports")

# List CSV files
csv_files = [file for file in os.listdir() if file.endswith('.csv')]

# Read CSV files into data frames
data_frames = {}
for file in csv_files:
    data_frames[file] = pd.read_csv(file)

# Merge data frames
cx_export_merged_data = pd.concat(data_frames.values(), ignore_index=True)

# Print column names
print(cx_export_merged_data.columns.tolist())

# Specify call date columns
call_date_columns = ["Call_date_1", "Call_date_2", "Call_date_3", "Call_date_4", "Call_date_5"]

# Create 'call_attempts' column
cx_export_merged_data['call_attempts'] = cx_export_merged_data[call_date_columns].apply(
    lambda row: row.dropna().count(), axis=1)

# Convert 'survey_id' to numeric
cx_export_merged_data['survey_id'] = pd.to_numeric(cx_export_merged_data['survey_id'], errors='coerce')

# Define function to determine call status
def determine_call_status(row):
    outcomes = [row[f'Call_outcome_{i}'] for i in range(1, 6)]
    if all(pd.isna(outcome) for outcome in outcomes):
        return "Not Called"
    elif any(outcome in outcomes for outcome in ["Fully Verified", "Partially Verified", "Not willing to participate"]):
        return outcomes[0]
    elif any(outcome in outcomes for outcome in ["Wrong Number", "Incorrect Data"]):
        return "Inaccurate Data"
    elif "Call Back Later" in outcomes:
        return "Call Back Later"
    elif "Phone Unanswered" in outcomes:
        return "Phone Unanswered"
    elif "Number Switched Off" in outcomes:
        return "Number Switched Off"
    else:
        return "Customer Unreachable"

# Create 'call_status' column
cx_export_merged_data['call_status'] = cx_export_merged_data.apply(determine_call_status, axis=1)


# Remove rows with country 'TEST_TANZANIA'
cx_export_merged_data = cx_export_merged_data[cx_export_merged_data['country'] != 'TEST_TANZANIA']




['survey_id', 'country', 'start', 'activator_label', 'activator_id', 'survey_region', 'survey_channel', 'sn_mod', 'product_name', 'name', 'main_contact', 'alternative_phone_label', 'residence_level1', 'residence_level2', 'residence_level3', 'preferred_time_to_be_called', 'preferred_language_to_be_called', 'Call_agent_email', 'Call_date_1', 'Call_date_2', 'Call_date_3', 'Call_date_4', 'Call_date_5', 'Call_outcome_1', 'Call_outcome_2', 'Call_outcome_3', 'Call_outcome_4', 'Call_outcome_5', 'Phone_number_of_the_respondent', 'What_is_the_name_of_the_primary_user', 'What_is_the_name_of_the_Region_State_the_stove_is_being_used', 'What_is_the_name_of_the_Ward_the_stove_is_being_used', 'Which_of_the_following_stoves_do_you_currently_use_for_most_of_your_meals', 'What_is_the_the_nearest_landmark_Village_Estate_avenue_school_churches', 'Where_did_you_meet_our_sales_representative_when_she_collected_your_details_for_this_purchase', 'Where_was_the_stove_delivered', 'What_fuel_do_you_use_most_in_you

In [22]:
print(cx_export_merged_data.columns)

Index(['survey_id', 'country', 'start', 'activator_label', 'activator_id',
       'survey_region', 'survey_channel', 'sn_mod', 'product_name', 'name',
       'main_contact', 'alternative_phone_label', 'residence_level1',
       'residence_level2', 'residence_level3', 'preferred_time_to_be_called',
       'preferred_language_to_be_called', 'Call_agent_email', 'Call_date_1',
       'Call_date_2', 'Call_date_3', 'Call_date_4', 'Call_date_5',
       'Call_outcome_1', 'Call_outcome_2', 'Call_outcome_3', 'Call_outcome_4',
       'Call_outcome_5', 'Phone_number_of_the_respondent',
       'What_is_the_name_of_the_primary_user',
       'What_is_the_name_of_the_Region_State_the_stove_is_being_used',
       'What_is_the_name_of_the_Ward_the_stove_is_being_used',
       'Which_of_the_following_stoves_do_you_currently_use_for_most_of_your_meals',
       'What_is_the_the_nearest_landmark_Village_Estate_avenue_school_churches',
       'Where_did_you_meet_our_sales_representative_when_she_collected_yo

In [23]:
# Convert date columns to datetime
date_columns = ['Date_created', 'start']
for col in date_columns:
    cx_export_merged_data[col] = pd.to_datetime(cx_export_merged_data[col], format="%Y-%m-%d", errors='coerce')

print(cx_export_merged_data)

           survey_id  country      start           activator_label  \
0        532542962.0  NIGERIA 2024-02-08             ZAINAB SALIHU   
1        532377022.0  NIGERIA 2024-02-08                 ABBA AKPA   
2        532407635.0  NIGERIA 2024-02-08                 ABBA AKPA   
3        532421228.0  NIGERIA 2024-02-08            REJOICE JOSHUA   
4        532421136.0  NIGERIA 2024-02-08            REJOICE JOSHUA   
...              ...      ...        ...                       ...   
1748077     147467.0    KENYA 2023-04-14                IMMACULATE   
1748078     140830.0    KENYA 2023-04-13            MOURICE OTWOMA   
1748079     134992.0    KENYA 2023-04-12                     ISAAC   
1748080     157871.0    KENYA 2023-04-15          MELLENIA MUNGOMA   
1748081     144076.0    KENYA 2023-04-13  HAPPINESS KEMUNTO ONDOSO   

        activator_id  survey_region survey_channel  sn_mod  product_name  \
0             CNL277    NIGER STATE            B2C  ABUV3Q  JIKOKOA PLUS   
1      

In [24]:
# Perform left join between cx_export_merged_data and df_registrations
cx_export_merged_data = pd.merge(cx_export_merged_data, 
                                 df_registrations[['sn_mod', 'verified_cc', 'type_of_business']], 
                                 on='sn_mod', 
                                 how='left')

# Print the joined data (which now retains the name cx_export_merged_data)
print(cx_export_merged_data)


           survey_id  country      start           activator_label  \
0        532542962.0  NIGERIA 2024-02-08             ZAINAB SALIHU   
1        532377022.0  NIGERIA 2024-02-08                 ABBA AKPA   
2        532407635.0  NIGERIA 2024-02-08                 ABBA AKPA   
3        532421228.0  NIGERIA 2024-02-08            REJOICE JOSHUA   
4        532421136.0  NIGERIA 2024-02-08            REJOICE JOSHUA   
...              ...      ...        ...                       ...   
1768598     147467.0    KENYA 2023-04-14                IMMACULATE   
1768599     140830.0    KENYA 2023-04-13            MOURICE OTWOMA   
1768600     134992.0    KENYA 2023-04-12                     ISAAC   
1768601     157871.0    KENYA 2023-04-15          MELLENIA MUNGOMA   
1768602     144076.0    KENYA 2023-04-13  HAPPINESS KEMUNTO ONDOSO   

        activator_id  survey_region survey_channel  sn_mod  product_name  \
0             CNL277    NIGER STATE            B2C  ABUV3Q  JIKOKOA PLUS   
1      

In [25]:
print(cx_export_merged_data.columns)

Index(['survey_id', 'country', 'start', 'activator_label', 'activator_id',
       'survey_region', 'survey_channel', 'sn_mod', 'product_name', 'name',
       'main_contact', 'alternative_phone_label', 'residence_level1',
       'residence_level2', 'residence_level3', 'preferred_time_to_be_called',
       'preferred_language_to_be_called', 'Call_agent_email', 'Call_date_1',
       'Call_date_2', 'Call_date_3', 'Call_date_4', 'Call_date_5',
       'Call_outcome_1', 'Call_outcome_2', 'Call_outcome_3', 'Call_outcome_4',
       'Call_outcome_5', 'Phone_number_of_the_respondent',
       'What_is_the_name_of_the_primary_user',
       'What_is_the_name_of_the_Region_State_the_stove_is_being_used',
       'What_is_the_name_of_the_Ward_the_stove_is_being_used',
       'Which_of_the_following_stoves_do_you_currently_use_for_most_of_your_meals',
       'What_is_the_the_nearest_landmark_Village_Estate_avenue_school_churches',
       'Where_did_you_meet_our_sales_representative_when_she_collected_yo

In [26]:
# Convert contents of 'call_status' column to uppercase
cx_export_merged_data['call_status'] = cx_export_merged_data['call_status'].str.upper()


In [27]:

# Drop duplicates based on 'sn_mod' and 'main_contact'
#cx_export_merged_data.drop_duplicates(subset=['sn_mod', 'main_contact'], inplace=True)
# Sort the data by 'call_attempts' column in ascending order
cx_export_merged_data.sort_values(by='call_attempts', ascending=True, inplace=True)

# Sort the data by 'call_status' column with specified order of preference
status_order = ['FULLY VERIFIED', 'PARTIALLY VERIFIED']
cx_export_merged_data['call_status'] = pd.Categorical(cx_export_merged_data['call_status'], categories=status_order, ordered=True)

# Drop duplicates based on 'sn_mod' and 'main_contact' while retaining rows according to specified conditions
cx_export_merged_data.sort_values(by=['sn_mod', 'main_contact', 'call_status'], inplace=True)
cx_export_merged_data.drop_duplicates(subset=['sn_mod', 'main_contact'], keep='last', inplace=True)



In [28]:
print("Number of rows in cx_export_merged_data:", cx_export_merged_data.shape[0])


Number of rows in cx_export_merged_data: 67416


In [29]:
##Nig Charcoal
import pandas as pd
import numpy as np

# Assuming 'submission_time' is a datetime column in your DataFrame cx_export_merged_data
cx_export_merged_data['Date_created'] = pd.to_datetime(cx_export_merged_data['Date_created'])

# Get the current year and month
current_year = pd.Timestamp.now().year
current_month = pd.Timestamp.now().month

# Filter DataFrame for the current month, type_of_business = CHARCOAL, and country = NIGERIA
cx_nigeria_char_current_month = cx_export_merged_data[(cx_export_merged_data['Date_created'].dt.year == current_year) & 
                                                       (cx_export_merged_data['Date_created'].dt.month == current_month) &
                                                       (cx_export_merged_data['type_of_business'] == 'CHARCOAL') &
                                                       (cx_export_merged_data['country'] == 'NIGERIA')]

# Group by weekly intervals and calculate metrics
weekly_summary_nigchar = cx_nigeria_char_current_month.groupby(pd.Grouper(key='Date_created', freq='W')).agg(
    loaded_registrations=('main_contact', 'nunique'),
    verified_MTD=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    unreachable=('call_status', lambda x: (~x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT'])).sum()),
    yet_to_be_contacted=('verified_cc', lambda x: x.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(cx_nigeria_char_current_month['verified_cc'] != 'NOT CONTACTED') & (cx_nigeria_char_current_month['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT', 'DORMANT']).sum())
)

# Calculate verification rates as percentages and approximate to one decimal place
weekly_summary_nigchar['old_verification_rate'] = round((weekly_summary_nigchar['verified_MTD'] / weekly_summary_nigchar['total_verified']) * 100, 1)
weekly_summary_nigchar['new_verification_rate'] = round((weekly_summary_nigchar['fully_verified'] / weekly_summary_nigchar['customer_called']) * 100, 1)

# Manually create the 'WEEK' column
weekly_summary_nigchar['WEEK'] = ['week' + str(i) for i in range(1, len(weekly_summary_nigchar) + 1)]

# Reorder columns to place 'WEEK' as the first column
weekly_summary_nigchar = weekly_summary_nigchar[['WEEK'] + [col for col in weekly_summary_nigchar.columns if col != 'WEEK']]

# Calculate sum totals for all metrics except 'verification_rate' columns and append the totals row to the DataFrame
sum_totals = weekly_summary_nigchar.drop(columns=['old_verification_rate', 'new_verification_rate', 'WEEK']).sum().to_frame().T
sum_totals['old_verification_rate'] = weekly_summary_nigchar['old_verification_rate'].mean()
sum_totals['new_verification_rate'] = weekly_summary_nigchar['new_verification_rate'].mean()
sum_totals['WEEK'] = 'Total'
weekly_summary_nigchar = weekly_summary_nigchar.append(sum_totals, ignore_index=True)

# Round all values in the total row to the nearest whole number
weekly_summary_nigchar = weekly_summary_nigchar.round()

# Convert 'verification_rate' columns to strings and add '%' sign
weekly_summary_nigchar['old_verification_rate'] = weekly_summary_nigchar['old_verification_rate'].astype(str) + '%'
weekly_summary_nigchar['new_verification_rate'] = weekly_summary_nigchar['new_verification_rate'].astype(str) + '%'

# Adding a title to the DataFrame
weekly_summary_nigchar_title = "Charcoal Summary for Nigeria MTD"
weekly_summary_nigchar = pd.concat([pd.DataFrame([weekly_summary_nigchar_title], columns=['Title']), weekly_summary_nigchar], ignore_index=True)

# Display the weekly summary
print(weekly_summary_nigchar)




                              Title   WEEK  loaded_registrations  \
0  Charcoal Summary for Nigeria MTD    NaN                   NaN   
1                               NaN  week1                  52.0   
2                               NaN  week2                1125.0   
3                               NaN  week3                 318.0   
4                               NaN  week4                 672.0   
5                               NaN  week5                 310.0   
6                               NaN  Total                2477.0   

   verified_MTD  unreachable  yet_to_be_contacted  fully_verified  \
0           NaN          NaN                  NaN             NaN   
1          46.0         29.0                  0.0            46.0   
2         990.0        610.0                  0.0           980.0   
3         244.0        266.0                  0.0           239.0   
4         476.0        633.0                  6.0           445.0   
5          38.0        300.0             

C:\Users\basil.chinedu\AppData\Local\Temp\10\ipykernel_55092\3669961878.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weekly_summary_nigchar = weekly_summary_nigchar.append(sum_totals, ignore_index=True)


In [30]:
##NIG WOOD
import pandas as pd
import numpy as np

# Assuming 'submission_time' is a datetime column in your DataFrame cx_export_merged_data
cx_export_merged_data['Date_created'] = pd.to_datetime(cx_export_merged_data['Date_created'])

# Get the current year and month
current_year = pd.Timestamp.now().year
current_month = pd.Timestamp.now().month

# Filter DataFrame for the current month, type_of_business = WOOD, and country = NIGERIA
cx_nigeria_wood_current_month = cx_export_merged_data[(cx_export_merged_data['Date_created'].dt.year == current_year) & 
                                                       (cx_export_merged_data['Date_created'].dt.month == current_month) &
                                                       (cx_export_merged_data['type_of_business'] == 'WOOD') &
                                                       (cx_export_merged_data['country'] == 'NIGERIA')]

# Group by weekly intervals and calculate metrics
weekly_summary_nigwood = cx_nigeria_wood_current_month.groupby(pd.Grouper(key='Date_created', freq='W')).agg(
    loaded_registrations=('main_contact', 'nunique'),
    verified_MTD=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    unreachable=('call_status', lambda x: (~x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT'])).sum()),
    yet_to_be_contacted=('verified_cc', lambda x: x.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(cx_nigeria_wood_current_month['verified_cc'] != 'NOT CONTACTED') & (cx_nigeria_wood_current_month['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT', 'DORMANT']).sum())
)

# Calculate verification rates as percentages and approximate to one decimal place
weekly_summary_nigwood['old_verification_rate'] = round((weekly_summary_nigwood['verified_MTD'] / weekly_summary_nigwood['total_verified']) * 100, 1)
weekly_summary_nigwood['new_verification_rate'] = round((weekly_summary_nigwood['fully_verified'] / weekly_summary_nigwood['customer_called']) * 100, 1)

# Manually create the 'WEEK' column
weekly_summary_nigwood['WEEK'] = ['week' + str(i) for i in range(1, len(weekly_summary_nigwood) + 1)]

# Reorder columns to place 'WEEK' as the first column
weekly_summary_nigwood = weekly_summary_nigwood[['WEEK'] + [col for col in weekly_summary_nigwood.columns if col != 'WEEK']]

# Calculate sum totals for all metrics except 'verification_rate' columns and append the totals row to the DataFrame
sum_totals = weekly_summary_nigwood.drop(columns=['old_verification_rate', 'new_verification_rate', 'WEEK']).sum().to_frame().T
sum_totals['old_verification_rate'] = weekly_summary_nigwood['old_verification_rate'].mean()
sum_totals['new_verification_rate'] = weekly_summary_nigwood['new_verification_rate'].mean()
sum_totals['WEEK'] = 'Total'
weekly_summary_nigwood = weekly_summary_nigwood.append(sum_totals, ignore_index=True)

# Round all values in the total row to the nearest whole number
weekly_summary_nigwood = weekly_summary_nigwood.round()

# Convert 'verification_rate' columns to strings and add '%' sign
weekly_summary_nigwood['old_verification_rate'] = weekly_summary_nigwood['old_verification_rate'].astype(str) + '%'
weekly_summary_nigwood['new_verification_rate'] = weekly_summary_nigwood['new_verification_rate'].astype(str) + '%'

# Adding a title to the DataFrame
weekly_summary_nigwood_title = "Wood Summary for Nigeria MTD"
weekly_summary_nigwood = pd.concat([pd.DataFrame([weekly_summary_nigwood_title], columns=['Title']), weekly_summary_nigwood], ignore_index=True)

# Display the weekly summary
print(weekly_summary_nigwood)


                          Title   WEEK  loaded_registrations  verified_MTD  \
0  Wood Summary for Nigeria MTD    NaN                   NaN           NaN   
1                           NaN  week1                   1.0           1.0   
2                           NaN  week2                 885.0         761.0   
3                           NaN  week3                1744.0        1054.0   
4                           NaN  week4                1292.0         532.0   
5                           NaN  week5                 666.0          22.0   
6                           NaN  Total                4588.0        2370.0   

   unreachable  yet_to_be_contacted  fully_verified  customer_called  \
0          NaN                  NaN             NaN              NaN   
1          1.0                  0.0             1.0              1.0   
2        502.0                  0.0           750.0            885.0   
3       1616.0                  0.0           996.0           1744.0   
4       1274.0 

C:\Users\basil.chinedu\AppData\Local\Temp\10\ipykernel_55092\158067736.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weekly_summary_nigwood = weekly_summary_nigwood.append(sum_totals, ignore_index=True)


In [31]:
import pandas as pd
import numpy as np

# Assuming 'submission_time' is a datetime column in your DataFrame cx_export_merged_data
cx_export_merged_data['Date_created'] = pd.to_datetime(cx_export_merged_data['Date_created'])

# Get the current year and month
current_year = pd.Timestamp.now().year
current_month = pd.Timestamp.now().month

# Filter DataFrame for the current month, type_of_business = CHARCOAL, and country = DRC
cx_nigeria_char_current_month = cx_export_merged_data[(cx_export_merged_data['Date_created'].dt.year == current_year) & 
                                                       (cx_export_merged_data['Date_created'].dt.month == current_month) &
                                                       (cx_export_merged_data['type_of_business'] == 'CHARCOAL') &
                                                       (cx_export_merged_data['country'] == 'DRC')]

# Group by weekly intervals and calculate metrics
weekly_summary_drcchar = cx_nigeria_char_current_month.groupby(pd.Grouper(key='Date_created', freq='W')).agg(
    loaded_registrations=('main_contact', 'nunique'),
    verified_MTD=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    unreachable=('call_status', lambda x: (~x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT'])).sum()),
    yet_to_be_contacted=('verified_cc', lambda x: x.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(cx_nigeria_char_current_month['verified_cc'] != 'NOT CONTACTED') & (cx_nigeria_char_current_month['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT', 'DORMANT']).sum())
)

# Calculate verification rates as percentages and approximate to one decimal place
weekly_summary_drcchar['old_verification_rate'] = round((weekly_summary_drcchar['verified_MTD'] / weekly_summary_drcchar['total_verified']) * 100, 1)
weekly_summary_drcchar['new_verification_rate'] = round((weekly_summary_drcchar['fully_verified'] / weekly_summary_drcchar['customer_called']) * 100, 1)

# Manually create the 'WEEK' column
weekly_summary_drcchar['WEEK'] = ['week' + str(i) for i in range(1, len(weekly_summary_drcchar) + 1)]

# Reorder columns to place 'WEEK' as the first column
weekly_summary_drcchar = weekly_summary_drcchar[['WEEK'] + [col for col in weekly_summary_drcchar.columns if col != 'WEEK']]

# Calculate sum totals for all metrics except 'verification_rate' columns and append the totals row to the DataFrame
sum_totals = weekly_summary_drcchar.drop(columns=['old_verification_rate', 'new_verification_rate', 'WEEK']).sum().to_frame().T
sum_totals['old_verification_rate'] = weekly_summary_drcchar['old_verification_rate'].mean()
sum_totals['new_verification_rate'] = weekly_summary_drcchar['new_verification_rate'].mean()
sum_totals['WEEK'] = 'Total'
weekly_summary_drcchar = weekly_summary_drcchar.append(sum_totals, ignore_index=True)

# Round all values in the total row to the nearest whole number
weekly_summary_drcchar = weekly_summary_drcchar.round()

# Convert 'verification_rate' columns to strings and add '%' sign
weekly_summary_drcchar['old_verification_rate'] = weekly_summary_drcchar['old_verification_rate'].astype(str) + '%'
weekly_summary_drcchar['new_verification_rate'] = weekly_summary_drcchar['new_verification_rate'].astype(str) + '%'

# Adding a title to the DataFrame
weekly_summary_drcchar_title = "Charcoal Summary for DRC MTD"
weekly_summary_drcchar = pd.concat([pd.DataFrame([weekly_summary_drcchar_title], columns=['Title']), weekly_summary_drcchar], ignore_index=True)

# Display the weekly summary
print(weekly_summary_drcchar)



                          Title   WEEK  loaded_registrations  verified_MTD  \
0  Charcoal Summary for DRC MTD    NaN                   NaN           NaN   
1                           NaN  week1                 339.0         132.0   
2                           NaN  week2                1625.0         680.0   
3                           NaN  week3                1538.0         617.0   
4                           NaN  week4                1128.0         404.0   
5                           NaN  week5                 326.0          53.0   
6                           NaN  Total                4956.0        1886.0   

   unreachable  yet_to_be_contacted  fully_verified  customer_called  \
0          NaN                  NaN             NaN              NaN   
1        232.0                  0.0            86.0            339.0   
2       1160.0                  0.0           413.0           1625.0   
3       1445.0                  3.0           396.0           1536.0   
4       1037.0 

C:\Users\basil.chinedu\AppData\Local\Temp\10\ipykernel_55092\2656049561.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weekly_summary_drcchar = weekly_summary_drcchar.append(sum_totals, ignore_index=True)


In [32]:
##MOZAMBIQUE CHARCOAL
import pandas as pd
import numpy as np

# Assuming 'submission_time' is a datetime column in your DataFrame cx_export_merged_data
cx_export_merged_data['Date_created'] = pd.to_datetime(cx_export_merged_data['Date_created'])

# Get the current year and month
current_year = pd.Timestamp.now().year
current_month = pd.Timestamp.now().month

# Filter DataFrame for the current month, type_of_business = CHARCOAL, and country = MOZAMBIQUE
cx_mozambique_char_current_month = cx_export_merged_data[(cx_export_merged_data['Date_created'].dt.year == current_year) & 
                                                          (cx_export_merged_data['Date_created'].dt.month == current_month) &
                                                          (cx_export_merged_data['type_of_business'] == 'CHARCOAL') &
                                                          (cx_export_merged_data['country'] == 'MOZAMBIQUE')]

# Group by weekly intervals and calculate metrics
weekly_summary_mozambique_char = cx_mozambique_char_current_month.groupby(pd.Grouper(key='Date_created', freq='W')).agg(
    loaded_registrations=('main_contact', 'nunique'),
    verified_MTD=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    unreachable=('call_status', lambda x: (~x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT'])).sum()),
    yet_to_be_contacted=('verified_cc', lambda x: x.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(cx_mozambique_char_current_month['verified_cc'] != 'NOT CONTACTED') & (cx_mozambique_char_current_month['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT', 'DORMANT']).sum())
)

# Calculate verification rates as percentages and approximate to one decimal place
weekly_summary_mozambique_char['old_verification_rate'] = round((weekly_summary_mozambique_char['verified_MTD'] / weekly_summary_mozambique_char['total_verified']) * 100, 1)
weekly_summary_mozambique_char['new_verification_rate'] = round((weekly_summary_mozambique_char['fully_verified'] / weekly_summary_mozambique_char['customer_called']) * 100, 1)

# Manually create the 'WEEK' column
weekly_summary_mozambique_char['WEEK'] = ['week' + str(i) for i in range(1, len(weekly_summary_mozambique_char) + 1)]

# Reorder columns to place 'WEEK' as the first column
weekly_summary_mozambique_char = weekly_summary_mozambique_char[['WEEK'] + [col for col in weekly_summary_mozambique_char.columns if col != 'WEEK']]

# Calculate sum totals for all metrics except 'verification_rate' columns and append the totals row to the DataFrame
sum_totals = weekly_summary_mozambique_char.drop(columns=['old_verification_rate', 'new_verification_rate', 'WEEK']).sum().to_frame().T
sum_totals['old_verification_rate'] = weekly_summary_mozambique_char['old_verification_rate'].mean()
sum_totals['new_verification_rate'] = weekly_summary_mozambique_char['new_verification_rate'].mean()
sum_totals['WEEK'] = 'Total'
weekly_summary_mozambique_char = weekly_summary_mozambique_char.append(sum_totals, ignore_index=True)

# Round all values in the total row to the nearest whole number
weekly_summary_mozambique_char = weekly_summary_mozambique_char.round()

# Convert 'verification_rate' columns to strings and add '%' sign
weekly_summary_mozambique_char['old_verification_rate'] = weekly_summary_mozambique_char['old_verification_rate'].astype(str) + '%'
weekly_summary_mozambique_char['new_verification_rate'] = weekly_summary_mozambique_char['new_verification_rate'].astype(str) + '%'

# Adding a title to the DataFrame
weekly_summary_mozambique_char_title = "Charcoal Summary for Mozambique MTD"
weekly_summary_mozambique_char = pd.concat([pd.DataFrame([weekly_summary_mozambique_char_title], columns=['Title']), weekly_summary_mozambique_char], ignore_index=True)

# Display the weekly summary
print(weekly_summary_mozambique_char)



                                 Title   WEEK  loaded_registrations  \
0  Charcoal Summary for Mozambique MTD    NaN                   NaN   
1                                  NaN  week1                  35.0   
2                                  NaN  week2                 128.0   
3                                  NaN  week3                 488.0   
4                                  NaN  week4                 777.0   
5                                  NaN  week5                 367.0   
6                                  NaN  Total                1795.0   

   verified_MTD  unreachable  yet_to_be_contacted  fully_verified  \
0           NaN          NaN                  NaN             NaN   
1          23.0         19.0                  0.0            19.0   
2          92.0         57.0                  1.0            69.0   
3         321.0        335.0                  1.0           228.0   
4         427.0        616.0                  0.0           234.0   
5          92.0  

C:\Users\basil.chinedu\AppData\Local\Temp\10\ipykernel_55092\3677717409.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weekly_summary_mozambique_char = weekly_summary_mozambique_char.append(sum_totals, ignore_index=True)


In [33]:
import pandas as pd
import numpy as np

# Assuming 'submission_time' is a datetime column in your DataFrame cx_export_merged_data
cx_export_merged_data['Date_created'] = pd.to_datetime(cx_export_merged_data['Date_created'])

# Get the current year and month
current_year = pd.Timestamp.now().year
current_month = pd.Timestamp.now().month

# Filter DataFrame for the current month, type_of_business = WOOD, and country = MOZAMBIQUE
cx_mozambique_wood_current_month = cx_export_merged_data[(cx_export_merged_data['Date_created'].dt.year == current_year) & 
                                                          (cx_export_merged_data['Date_created'].dt.month == current_month) &
                                                          (cx_export_merged_data['type_of_business'] == 'WOOD') &
                                                          (cx_export_merged_data['country'] == 'MOZAMBIQUE')]

# Group by weekly intervals and calculate metrics
weekly_summary_mozambique_wood = cx_mozambique_wood_current_month.groupby(pd.Grouper(key='Date_created', freq='W')).agg(
    loaded_registrations=('main_contact', 'nunique'),
    verified_MTD=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    unreachable=('call_status', lambda x: (~x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED','PARTIALLY VERIFIED - DORMANT'])).sum()),
    yet_to_be_contacted=('verified_cc', lambda x: x.isin(['NOT CONTACTED', 'SAMPLED - NOT CONTACTED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(cx_mozambique_wood_current_month['verified_cc'] != 'NOT CONTACTED') & (cx_mozambique_wood_current_month['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT', 'DORMANT']).sum())
)

# Calculate verification rates as percentages and approximate to one decimal place
weekly_summary_mozambique_wood['old_verification_rate'] = round((weekly_summary_mozambique_wood['verified_MTD'] / weekly_summary_mozambique_wood['total_verified']) * 100, 1)
weekly_summary_mozambique_wood['new_verification_rate'] = round((weekly_summary_mozambique_wood['fully_verified'] / weekly_summary_mozambique_wood['customer_called']) * 100, 1)

# Manually create the 'WEEK' column
weekly_summary_mozambique_wood['WEEK'] = ['week' + str(i) for i in range(1, len(weekly_summary_mozambique_wood) + 1)]

# Reorder columns to place 'WEEK' as the first column
weekly_summary_mozambique_wood = weekly_summary_mozambique_wood[['WEEK'] + [col for col in weekly_summary_mozambique_wood.columns if col != 'WEEK']]

# Calculate sum totals for all metrics except 'verification_rate' columns and append the totals row to the DataFrame
sum_totals = weekly_summary_mozambique_wood.drop(columns=['old_verification_rate', 'new_verification_rate', 'WEEK']).sum().to_frame().T
sum_totals['old_verification_rate'] = weekly_summary_mozambique_wood['old_verification_rate'].mean()
sum_totals['new_verification_rate'] = weekly_summary_mozambique_wood['new_verification_rate'].mean()
sum_totals['WEEK'] = 'Total'
weekly_summary_mozambique_wood = weekly_summary_mozambique_wood.append(sum_totals, ignore_index=True)

# Round all values in the total row to the nearest whole number
weekly_summary_mozambique_wood = weekly_summary_mozambique_wood.round()

# Convert 'verification_rate' columns to strings and add '%' sign
weekly_summary_mozambique_wood['old_verification_rate'] = weekly_summary_mozambique_wood['old_verification_rate'].astype(str) + '%'
weekly_summary_mozambique_wood['new_verification_rate'] = weekly_summary_mozambique_wood['new_verification_rate'].astype(str) + '%'

# Adding a title to the DataFrame
weekly_summary_mozambique_wood_title = "Wood Summary for Mozambique MTD"
weekly_summary_mozambique_wood = pd.concat([pd.DataFrame([weekly_summary_mozambique_wood_title], columns=['Title']), weekly_summary_mozambique_wood], ignore_index=True)

# Display the weekly summary
print(weekly_summary_mozambique_wood)


                             Title   WEEK  loaded_registrations  verified_MTD  \
0  Wood Summary for Mozambique MTD    NaN                   NaN           NaN   
1                              NaN  week1                 267.0         185.0   
2                              NaN  week2                 120.0          68.0   
3                              NaN  week3                  58.0          33.0   
4                              NaN  week4                  17.0           8.0   
5                              NaN  week5                  22.0           1.0   
6                              NaN  Total                 484.0         295.0   

   unreachable  yet_to_be_contacted  fully_verified  customer_called  \
0          NaN                  NaN             NaN              NaN   
1        146.0                  0.0           121.0            267.0   
2         68.0                  1.0            47.0            119.0   
3         31.0                  0.0            24.0            

C:\Users\basil.chinedu\AppData\Local\Temp\10\ipykernel_55092\1117416424.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weekly_summary_mozambique_wood = weekly_summary_mozambique_wood.append(sum_totals, ignore_index=True)


In [34]:
# Daily Analysis MTD
import pandas as pd
from datetime import datetime

# Filter data for current month
current_month = datetime.now().month
current_year = datetime.now().year

df_verifications_current_month = df_verifications[df_verifications['submission_time'].dt.month == current_month]
df_verifications_current_month = df_verifications_current_month[df_verifications_current_month['submission_time'].dt.year == current_year]

df_registrations_current_month = df_registrations[df_registrations['submission_time'].dt.month == current_month]
df_registrations_current_month = df_registrations_current_month[df_registrations_current_month['submission_time'].dt.year == current_year]

# Calculate total registrations by submission_time and country
total_registrations = df_registrations_current_month.groupby(['submission_time', 'country'])['main_contact'].nunique().reset_index()

# Summarize registrations by submission_time and country
summary_registrations_daily_analysis = df_verifications_current_month.groupby(['submission_time', 'country']).agg(
    sample_pasted=('main_contact', 'nunique'),
    verified_registrations=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    partially_verified=('verified_cc', lambda x: x.isin(['PARTIALLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(df_verifications_current_month['verified_cc'] != 'NOT CONTACTED') & (df_verifications_current_month['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_calls_made=('call_attempts', lambda x: ((df_verifications_current_month['verified_cc'] != 'NOT CONTACTED') & (df_verifications_current_month['verified_cc'] != 'SAMPLED - NOT CONTACTED') & (~x.isnull())).sum()),
    customers_reached=('call_status', lambda x: x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT']).sum()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT', 'DORMANT']).sum())
).reset_index()



# Calculate total registrations by submission_time and country
total_registrations_by_submission_time_country = df_registrations_current_month.groupby(['submission_time', 'country'])['main_contact'].nunique().reset_index()
total_registrations_by_submission_time_country.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Merge total registrations by submission_time and country with summary_registrations_daily_analysis
summary_registrations_daily_analysis = pd.merge(summary_registrations_daily_analysis, total_registrations_by_submission_time_country, on=['submission_time', 'country'], how='left')

# Calculate additional metrics
summary_registrations_daily_analysis['old_verification_rate'] = (summary_registrations_daily_analysis['verified_registrations'] / summary_registrations_daily_analysis['total_verified']) * 100
summary_registrations_daily_analysis['new_verification_rate'] = (summary_registrations_daily_analysis['fully_verified'] / summary_registrations_daily_analysis['customer_called']) * 100
summary_registrations_daily_analysis['sample_percentage'] = (summary_registrations_daily_analysis['sample_pasted'] / summary_registrations_daily_analysis['total_registrations']) * 100
summary_registrations_daily_analysis['call_rate'] = (summary_registrations_daily_analysis['customer_called'] / summary_registrations_daily_analysis['sample_pasted']) * 100
summary_registrations_daily_analysis['reachability_rate'] = (summary_registrations_daily_analysis['customers_reached'] / summary_registrations_daily_analysis['customer_called']) * 100
summary_registrations_daily_analysis['percentage_fully_verified'] = (summary_registrations_daily_analysis['fully_verified'] / summary_registrations_daily_analysis['sample_pasted']) * 100


# Assuming your DataFrame is named summary_registrations_daily_analysis

# Selecting the desired columns
selected_columns = ['submission_time', 'country', 'total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'old_verification_rate', 'new_verification_rate', 'customer_called', 'total_calls_made', 'call_rate', 'customers_reached', 'reachability_rate', 'fully_verified', 'partially_verified', 'percentage_fully_verified']

# Reordering the columns
summary_registrations_daily_analysis = summary_registrations_daily_analysis[selected_columns]

# Displaying the DataFrame
print(summary_registrations_daily_analysis)




    submission_time     country  total_registrations  sample_pasted  \
0        2024-03-01         DRC                  328            239   
1        2024-03-01       KENYA                   33             21   
2        2024-03-01  MOZAMBIQUE                   20             14   
3        2024-03-01    TANZANIA                   43             41   
4        2024-03-02         DRC                  114             70   
..              ...         ...                  ...            ...   
129      2024-03-27         DRC                  369            195   
130      2024-03-27       KENYA                  486            200   
131      2024-03-27  MOZAMBIQUE                  224            224   
132      2024-03-27     NIGERIA                  411            109   
133      2024-03-27    TANZANIA                  253            134   

     sample_percentage  verified_registrations  old_verification_rate  \
0            72.865854                      82              88.172043   
1

In [35]:
import pandas as pd
from datetime import datetime

# Filter data for current month
current_month = datetime.now().month
current_year = datetime.now().year

df_verifications_current_month = df_verifications[df_verifications['submission_time'].dt.month == current_month]
df_verifications_current_month = df_verifications_current_month[df_verifications_current_month['submission_time'].dt.year == current_year]

df_registrations_current_month = df_registrations[df_registrations['submission_time'].dt.month == current_month]
df_registrations_current_month = df_registrations_current_month[df_registrations_current_month['submission_time'].dt.year == current_year]

# Compute total registrations by submission time and country
total_registrations_by_submission_time_country = df_registrations_current_month.groupby(['submission_time', 'activator_label', 'activator_id', 'country'])['main_contact'].nunique().reset_index()
total_registrations_by_submission_time_country.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Summarize registrations by submission time, country, and other columns
summary_registrations_agent_daily_analysis = df_verifications_current_month.groupby(['submission_time', 'activator_label', 'activator_id', 'country']).agg(
    sample_pasted=('main_contact', 'nunique'),
    verified_registrations=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    partially_verified=('verified_cc', lambda x: x.isin(['PARTIALLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(df_verifications_current_month['verified_cc'] != 'NOT CONTACTED') & (df_verifications_current_month['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_calls_made=('call_attempts', lambda x: ((df_verifications_current_month['verified_cc'] != 'NOT CONTACTED') & (df_verifications_current_month['verified_cc'] != 'SAMPLED - NOT CONTACTED') & (~x.isnull())).sum()),
    customers_reached=('call_status', lambda x: x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT']).sum()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT', 'DORMANT']).sum())
).reset_index()

# Merge total registrations by submission time and country with summary_registrations_daily_analysis
summary_registrations_agent_daily_analysis = pd.merge(summary_registrations_agent_daily_analysis, total_registrations_by_submission_time_country, on=['submission_time', 'activator_label', 'activator_id', 'country'], how='left')

# Calculate additional metrics
summary_registrations_agent_daily_analysis['old_verification_rate'] = (summary_registrations_agent_daily_analysis['verified_registrations'] / summary_registrations_agent_daily_analysis['total_verified']) * 100
summary_registrations_agent_daily_analysis['new_verification_rate'] = (summary_registrations_agent_daily_analysis['fully_verified'] / summary_registrations_agent_daily_analysis['customer_called']) * 100
summary_registrations_agent_daily_analysis['sample_percentage'] = (summary_registrations_agent_daily_analysis['sample_pasted'] / summary_registrations_agent_daily_analysis['total_registrations']) * 100
summary_registrations_agent_daily_analysis['call_rate'] = (summary_registrations_agent_daily_analysis['customer_called'] / summary_registrations_agent_daily_analysis['sample_pasted']) * 100
summary_registrations_agent_daily_analysis['reachability_rate'] = (summary_registrations_agent_daily_analysis['customers_reached'] / summary_registrations_agent_daily_analysis['customer_called']) * 100
summary_registrations_agent_daily_analysis['percentage_fully_verified'] = (summary_registrations_agent_daily_analysis['fully_verified'] / summary_registrations_agent_daily_analysis['sample_pasted']) * 100


# Reorder columns
summary_registrations_agent_daily_analysis = summary_registrations_agent_daily_analysis[['submission_time', 'country', 'total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'old_verification_rate', 'new_verification_rate', 'customer_called', 'total_calls_made', 'call_rate', 'customers_reached', 'reachability_rate', 'fully_verified', 'partially_verified', 'percentage_fully_verified']]

# Display DataFrame
print(summary_registrations_agent_daily_analysis)



     submission_time     country  total_registrations  sample_pasted  \
0         2024-03-01  MOZAMBIQUE                  NaN              1   
1         2024-03-01         DRC                 15.0             15   
2         2024-03-01         DRC                  9.0              9   
3         2024-03-01    TANZANIA                  1.0              1   
4         2024-03-01         DRC                  3.0              3   
...              ...         ...                  ...            ...   
7267      2024-03-27    TANZANIA                  2.0              2   
7268      2024-03-27       KENYA                  2.0              2   
7269      2024-03-27       KENYA                  1.0              1   
7270      2024-03-27  MOZAMBIQUE                  1.0              1   
7271      2024-03-27  MOZAMBIQUE                  1.0              1   

      sample_percentage  verified_registrations  old_verification_rate  \
0                   NaN                       0              

In [36]:
import pandas as pd
from datetime import datetime

# Filter data for current month
current_month = datetime.now().month
current_year = datetime.now().year

df_verifications_current_month = df_verifications[df_verifications['submission_time'].dt.month == current_month]
df_verifications_current_month = df_verifications_current_month[df_verifications_current_month['submission_time'].dt.year == current_year]

df_registrations_current_month = df_registrations[df_registrations['submission_time'].dt.month == current_month]
df_registrations_current_month = df_registrations_current_month[df_registrations_current_month['submission_time'].dt.year == current_year]

# Calculate distinct count of main contacts from df_registrations by country, activator_id, and activator_label
total_registrations = df_registrations_current_month.groupby(['country', 'activator_id', 'activator_label'])['main_contact'].nunique().reset_index()

# Summarize registrations by country, activator_id, and activator_label
Agent_Summary_Analysis = df_verifications_current_month.groupby(['country', 'activator_id', 'activator_label']).agg(
    sample_pasted=('main_contact', 'nunique'),
    verified_registrations=('verified_cc', lambda x: x.isin(['FULLY VERIFIED', 'PARTIALLY VERIFIED']).sum()),
    fully_verified=('verified_cc', lambda x: x.isin(['FULLY VERIFIED']).sum()),
    partially_verified=('verified_cc', lambda x: x.isin(['PARTIALLY VERIFIED']).sum()),
    customer_called=('main_contact', lambda x: x[(df_verifications_current_month['verified_cc'] != 'NOT CONTACTED') & (df_verifications_current_month['verified_cc'] != 'SAMPLED - NOT CONTACTED')].nunique()),
    total_calls_made=('call_attempts', lambda x: ((df_verifications_current_month['verified_cc'] != 'NOT CONTACTED') & (df_verifications_current_month['verified_cc'] != 'SAMPLED - NOT CONTACTED') & (~x.isnull())).sum()),
    customers_reached=('call_status', lambda x: x.isin(['DECLINE', 'FOLLOW UP', 'REACHABLE', 'FULLY VERIFIED', 'VERIFIED', 'NOT WILLING TO PARTICIPATE', 'WRONG NUMBER', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT']).sum()),
    total_verified=('verified_cc', lambda x: x.isin(['DECLINE', 'DORMANT (HARD TO RESOLVE)', 'FULLY VERIFIED', 'DOUBTFULLY VERIFIED', 'WRONG DATA', 'PARTIALLY VERIFIED', 'PARTIALLY VERIFIED - DORMANT', 'DORMANT']).sum())
).reset_index()

# Calculate total registrations by country, activator_id, and activator_label
total_registrations_by_country_activator = df_registrations_current_month.groupby(['country', 'activator_id', 'activator_label'])['main_contact'].nunique().reset_index()
total_registrations_by_country_activator.rename(columns={'main_contact': 'total_registrations'}, inplace=True)

# Merge total registrations by country, activator_id, and activator_label with Agent_Summary_Analysis
Agent_Summary_Analysis = pd.merge(Agent_Summary_Analysis, total_registrations_by_country_activator, on=['country', 'activator_id', 'activator_label'], how='left')

# Calculate old verification rate
Agent_Summary_Analysis['old_verification_rate'] = (Agent_Summary_Analysis['verified_registrations'] / Agent_Summary_Analysis['total_verified']) * 100

####Calculate new verification rate
Agent_Summary_Analysis['new_verification_rate'] = (Agent_Summary_Analysis['fully_verified'] / Agent_Summary_Analysis['customer_called']) * 100
# Calculate sample percentage correctly
Agent_Summary_Analysis['sample_percentage'] = (Agent_Summary_Analysis['sample_pasted'] / Agent_Summary_Analysis['total_registrations']) * 100

# Calculate call rate correctly
Agent_Summary_Analysis['call_rate'] = (Agent_Summary_Analysis['customer_called'] / Agent_Summary_Analysis['sample_pasted']) * 100

# Calculate reachability_rate correctly
Agent_Summary_Analysis['reachability_rate'] = (Agent_Summary_Analysis['customers_reached'] / Agent_Summary_Analysis['customer_called']) * 100

# Calculate fully verified_rate correctly
Agent_Summary_Analysis['percentage_fully_verified'] = (Agent_Summary_Analysis['fully_verified'] / Agent_Summary_Analysis['sample_pasted']) * 100

# Assuming your DataFrame is named Agent_Summary_Analysis

# Selecting the desired columns
selected_columns = ['country', 'activator_id', 'activator_label', 'total_registrations', 'sample_pasted', 'sample_percentage', 'verified_registrations', 'old_verification_rate', 'new_verification_rate', 'customer_called', 'total_calls_made', 'call_rate', 'customers_reached', 'reachability_rate', 'fully_verified', 'partially_verified', 'percentage_fully_verified']

# Reordering the columns
Agent_Summary_Analysis = Agent_Summary_Analysis[selected_columns]

# Displaying the DataFrame
print(Agent_Summary_Analysis)



       country activator_id      activator_label  total_registrations  \
0          DRC       CDF002       MANSONI KETHIA                 96.0   
1          DRC       CDF023    ELIANE NDOLANDOLA                 18.0   
2          DRC       CDF024     TOUCHERIE NTUMBA                 34.0   
3          DRC       CDF045       BERTHIANA WEVA                 57.0   
4          DRC       CDF047       CHARITE MPONGO                  6.0   
...        ...          ...                  ...                  ...   
1619  TANZANIA          469        BENSON KYANDO                  4.0   
1620  TANZANIA          857        MWANAIDI DADY                 25.0   
1621  TANZANIA          877     HUSSEIN SULEIMAN                  1.0   
1622  TANZANIA          912  MAGDALENA MWANSHETO                 17.0   
1623  TANZANIA          952        FARAJI SHAFII                  6.0   

      sample_pasted  sample_percentage  verified_registrations  \
0                53          55.208333                   

In [37]:
print("Columns in DataFrame:")
print(cx_export_merged_data.columns)

# Check if 'modified_on' column exists in the DataFrame
if 'modified_on' in cx_export_merged_data.columns:
    print("The 'modified_on' column exists in the DataFrame.")
else:
    print("The 'modified_on' column does not exist in the DataFrame.")


Columns in DataFrame:
Index(['survey_id', 'country', 'start', 'activator_label', 'activator_id',
       'survey_region', 'survey_channel', 'sn_mod', 'product_name', 'name',
       'main_contact', 'alternative_phone_label', 'residence_level1',
       'residence_level2', 'residence_level3', 'preferred_time_to_be_called',
       'preferred_language_to_be_called', 'Call_agent_email', 'Call_date_1',
       'Call_date_2', 'Call_date_3', 'Call_date_4', 'Call_date_5',
       'Call_outcome_1', 'Call_outcome_2', 'Call_outcome_3', 'Call_outcome_4',
       'Call_outcome_5', 'Phone_number_of_the_respondent',
       'What_is_the_name_of_the_primary_user',
       'What_is_the_name_of_the_Region_State_the_stove_is_being_used',
       'What_is_the_name_of_the_Ward_the_stove_is_being_used',
       'Which_of_the_following_stoves_do_you_currently_use_for_most_of_your_meals',
       'What_is_the_the_nearest_landmark_Village_Estate_avenue_school_churches',
       'Where_did_you_meet_our_sales_representative

In [38]:
# Assuming 'modified_on' is the column containing datetime objects
cx_export_merged_data['Date_created'] = cx_export_merged_data['Date_created'].dt.tz_localize(None)

In [39]:
##########################################################################################################################
# Define the path to the directory
directory_path = r"D:\Burn Manufacturing\Marketing - Documents\Business Intelligence\BI Automations 2020\Basil\Sample Verification Report"

from datetime import datetime

# Define the file name with current date
current_date = datetime.today().strftime('%Y-%m-%d')
file_name = f"Sample Verification Report_{current_date}.xlsx"

# Concatenate the directory path and file name
file_path = fr"{directory_path}\{file_name}"




# Write to Excel file
with pd.ExcelWriter(file_path, engine="openpyxl", mode="w") as writer:
    metrics_df.to_excel(writer, sheet_name='Metrics and Measurements', index=False, startrow=1, startcol=0)
    summary_registrations.to_excel(writer, sheet_name='Sep-feb Verification by Country', index=False, startrow=1, startcol=0)
    summary_registrations_mar2024.to_excel(writer, sheet_name='Sep-feb Verification by Country', index=False, startrow=13, startcol=0)
    summary_registrations_feb2024.to_excel(writer, sheet_name='Sep-feb Verification by Country', index=False, startrow=25, startcol=0)
    summary_registrations_jan2024.to_excel(writer, sheet_name='Sep-feb Verification by Country', index=False, startrow=37, startcol=0) 
    summary_registrations_dec2023.to_excel(writer, sheet_name='Sep-feb Verification by Country', index=False, startrow=49, startcol=0) 
    summary_registrations_nov2023.to_excel(writer, sheet_name='Sep-feb Verification by Country', index=False, startrow=61, startcol=0) 
    summary_registrations_oct2023.to_excel(writer, sheet_name='Sep-feb Verification by Country', index=False, startrow=73, startcol=0) 
    summary_registrations_sept2023.to_excel(writer, sheet_name='Sep-feb Verification by Country', index=False, startrow=85, startcol=0)
    weekly_summary_nigchar.to_excel(writer, sheet_name='CX App Verification', index=False, startrow=2, startcol=0)
    weekly_summary_nigwood.to_excel(writer, sheet_name='CX App Verification', index=False, startrow=13, startcol=0)
    weekly_summary_drcchar.to_excel(writer, sheet_name='CX App Verification', index=False, startrow=24, startcol=0)
    weekly_summary_mozambique_char.to_excel(writer, sheet_name='CX App Verification', index=False, startrow=35, startcol=0)
    weekly_summary_mozambique_wood.to_excel(writer, sheet_name='CX App Verification', index=False, startrow=46, startcol=0)
    summary_registrations_daily_analysis.to_excel(writer, sheet_name='Daily Analysis MTD', index=False) 
    summary_registrations_agent_daily_analysis.to_excel(writer, sheet_name='Agent Daily Analysis MTD', index=False) 
    Agent_Summary_Analysis.to_excel(writer, sheet_name='Agents Summary Analysis MTD', index=False)








In [ ]:
import os
from email.mime.base import MIMEBase
from email import encoders

# Email configuration
# Date
today = date.today()  # Get the current date
# Format the date
formatted_date = today.strftime("%d %b %Y")

# Dataframe 
#email_df = result_df[['country','total_new_registrations','sample_registrations_pasted','Sample Percentage','Verification Rate','Call Rate',]]
# # Drop the index (resetting the index)
# email_df = email_df.reset_index(drop=True)

# Set the "country" column as the index
email_df = summary_registrations_mar2024.set_index('country')

# Round specific columns to the nearest whole numbers
email_df[['total_registrations', 'sample_pasted']] = email_df[['total_registrations', 'sample_pasted']].round()

# Select specific columns (if needed)
email_df = email_df[['total_registrations', 'sample_pasted', 'sample_percentage', 'old_verification_rate', 'new_verification_rate', 'call_rate']]

# Remove the second row from email_df
email_df = email_df.drop(email_df.index[0])

# Round specific columns to the nearest whole numbers and convert to integers
email_df['total_registrations'] = email_df['total_registrations'].round().astype(int)
email_df['sample_pasted'] = email_df['sample_pasted'].round().astype(int)


smtp_host = 'smtp.office365.com'
smtp_port = 587
sender_email = 'business.intelligence@burnmfg.com'
sender_password = 'Burn@twentytwenty01'
# receiver_emails = ['chris.mckinney@burnmfg.com','jacob.okoth@burnmfg.com','mark.connor@burnmfg.com','stephen.kingsley@burnmfg.com','abigail.kalua@burnmfg.com','etulan.ikpoki@burnmfg.com','irene.kamande@burnmfg.com','robert.mwakajwanga@burnmfg.com','akilimali.arsene@burnmfg.com','winifred.ndegwa@burnmfg.com','ida.ogembo@burnmfg.com','chidi.ohaji@burnmfg.com']
# cc_emails = ['samuel.njihia@burnmfg.com','antony.rono@burnmfg.com','eric.njiraini@burnmfg.com','francis.gichere@burnmfg.com','teresiah.karumba@burnmfg.com','chidi.ohaji@burnmfg.com','ian.juma@burnmfg.com','mercy.chege@burnmfg.com']
#receiver_emails = ['basil.chinedu@burnmfg.com']
#cc_emails = ['basil.chinedu@burnmfg.com']
receiver_emails = ['basil.chinedu@burnmfg.com', 'irene.obinikpo@burnmfg.com','ida.ogembo@burnmfg.com','abigail.kalua@burnmfg.com', 'chidi.ohaji@burnmfg.com', 'akilimali.arsene@burnmfg.com', 'winifred.ndegwa@burnmfg.com']
cc_emails = ['eric.njiraini@burnmfg.com','francis.gichere@burnmfg.com', 'antony.rono@burnmfg.com']

# Compose the email message
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = ', '.join(receiver_emails)  # Join multiple recipients with a comma
message['Cc'] = ', '.join(cc_emails)  # Join multiple CC recipients with a comma
message['Subject'] = 'Sample Verification Report Draft Which Captures The New Verification Rate as at ' + str(formatted_date)




# Create the HTML table with borders for the pivot tables
html = """\
<html>
  <head>
    <style>
      table {{
        border-collapse: collapse;
        width: 30%;
      }}
      th, td {{
        border: 1px solid black;
        padding: 5px;
        text-align: left;
      }}
       th{{
        background-color: #eaf2f5;  /* White */
      }}

      tr:last-child {{
        background-color: #eaf2f5;  /* White */
      }}
   
    </style>
  </head>
  <body>
  <p>Hi Team,</p>
  <p>The above attachment is the sample verification report as at March 2024 which includes the new sample verification rate</p>  
  <p>On a high level the report is as shown below: <p>
    {}

   <p> For any queries feel free to reach out via <a href='mailto:basil.chinedu@burnmfg.com'>basil.chinedu@burnmfg.com</a>.</p>
   <p> Regards,</p>
   <p> Basil Chinedu</p>
  </body>
</html>
""".format(email_df.to_html())

# Attach the HTML table to the email
message.attach(MIMEText(html, 'html'))

# Attachment
attachment_path = rf'D:\Burn Manufacturing\Marketing - Documents\Business Intelligence\BI Automations 2020\Basil\Sample Verification Report\Sample Verification Report_{current_date}.xlsx'

# Open the file in binary mode
with open(attachment_path, 'rb') as attachment:
    # Add file as application/octet-stream
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())

# Encode file in ASCII characters to send by email    
encoders.encode_base64(part)

# Add header as key/value pair to attachment part
part.add_header(
    'Content-Disposition',
    f'attachment; filename= {os.path.basename(attachment_path)}',
)

# Add attachment to message and convert message to string
message.attach(part)

#Connect to the SMTP server and send the email
with smtplib.SMTP(smtp_host, smtp_port) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    server.send_message(message)
